# Calculate GWP and create tables

The script calculates the GWP of hydrogen and methane using a steady-state perturbation approach. This build on Sand et al. 2023, and use the same method to calculate GWP for different sensitivity tests using the OsloCTM3 model.

**Sensitivity tests**:
- **Size of perturbation**
- **Location of pertubation**
- **Sensitivity of chemical background**


For all tests, a set of three simulations are used:
- **CTRL**: fixed emission of hydrogen and surface concentration of methane
- **pertH2**: as CTRL, but with surface H2 increased. Run to steady state.
- **10CH4**: as CTRL, but with surface CH4 increased by 10%. Run to steady state.


GWP is the ratio of the absolute global warming potential (AGWP) for hydrogen to that for CO2. AGWP is defined as the time-integrated radiative forcing to a 1 kg pulse emission of that gas over a given time horizon, here 100 years. 

AGWP is equal to the steady-state radiative forcing (W m-2) divided by the delta flux (Tg-H2 yr-1) for a perturbed vs control. This delta flux between the perturbed (10H2 or 10CH4) and the control simulation (CTRL) includes chemical feedbacks. 

Based on the three set of simulations, we calculate the radiative forcing in the perturbed relative to the control. As the models run with fixed surface concentration of methane, we need to have a separate perturbed methane run. We calculate the radiative forcing per Tg-CH4 (including the feedbacks, as feedbacks are included in the delta flux) for the methane perturbed run, and map the changes in the methane loss in the hydrogen perturbation with the results from the methane perturbed run. 


#### OUTLINE:
**PART I: Read model results**

**PART II: GWP calculations**

**PART III: Main results and tabels**

**Appendix with additional results**

**Uncertainty analysis output**

This is the main notebook for the GWP calculations. The uncertainty analysis and plotting are done in separate notebooks. 

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '{:,.4f}'.format(x) if abs(x)<0 else ('{:,.2f}'.format(x) if abs(x)<10 else ('{:,.1f}'.format(x) if abs(x)<100 else '{:,.0f}'.format(x))))

Input and output path:

In [2]:
path = r"./input/"
pathssp = r"./input_ssp/"
outputpath= r"./output/"

Constants:

In [3]:
#AGWP100_CO2 [mW yr m-2 Tg-1] Source: Table 7.SM.6 in IPCC AR6: 0.0895 pW m-2 yr kg-1 (p=10^-12) 
agwp100_CO2 = 0.0895

#CH4 tau_strat[yr] 
tau_strat = 120.0*2.0 #Multiplied by to for not double counting OH loss in stratosphere. 

#CH4 tau_soil [yr] 
tau_soil = 160.0

#Specific RF for CH4 [mW m-2 ppb-1] Etminan et al., 2016
spec_rf_ch4 = 0.44800

#The adjustment is –14% ± 15% IPCC AR6
spec_rf_ch4 = spec_rf_ch4*(1.0-0.14)
print(spec_rf_ch4)

0.38528


# Part I: Read model results

In this part, model results are read from the input files.

10H2 is the hydrogen perturbed simulations. To do: Rename to pertH2.

## 1. Hydrogen budget

### 1.1 H2 burden [Tg]:

In [4]:
file = 'H2_burden.txt'
df_h2_burden = pd.read_csv(path + file, sep=';',index_col=0,header=0)
delta = df_h2_burden.loc['10H2']-df_h2_burden.loc['CTRL']
delta.name = 'deltaH2'
df_h2_burden_conc = df_h2_burden.append(delta)
df_h2_burden_conc = df_h2_burden['OSLOCTM3']
df_h2_burden_conc

Scenario
CTRL    195
10H2    nan
10CH4   196
Name: OSLOCTM3, dtype: float64

In [5]:
file = 'OSLOCTM3-emi_burden-H2.csv'
df_h2_burden = pd.read_csv(path + file, sep=',',index_col=0,header=0)
#delta = df_h2_burden.loc['10H2']-df_h2_burden.loc['CTRL']
#delta.name = 'deltaH2'
df_h2_burden.loc['deltaH2'] = df_h2_burden.iloc[0]-df_h2_burden['CNTR'].iloc[0]
df_h2_burden
 

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
YR13,205,205,207,226,423,207,207,207,206,207,207,207
deltaH2,0.00,0.22,2.15,21.5,218,2.53,1.93,2.05,1.76,2.56,2.07,1.97


In [6]:
file = 'OSLOCTM3-emi_burden-H2.csv'
df_h2_burden_ssp = pd.read_csv(pathssp + file, sep=',',index_col=0,header=0)


ssps = ['SSP119','SSP434','SSP585']
df_h2_burden_ssp.loc['deltaH2'] = np.nan
for ssp in ssps:
    name = '10CH4_'+ssp
    df_h2_burden_ssp[name].loc['deltaH2'] = df_h2_burden_ssp[name].iloc[0]-df_h2_burden_ssp['CNTR_'+ssp].iloc[0]
    name = 'antro10_'+ssp
    df_h2_burden_ssp[name].loc['deltaH2'] = df_h2_burden_ssp[name].iloc[0]-df_h2_burden_ssp['CNTR_'+ssp].iloc[0]
    name = 'CNTR_'+ssp
    df_h2_burden_ssp[name].loc['deltaH2'] = df_h2_burden_ssp[name].iloc[0]-df_h2_burden_ssp['CNTR_'+ssp].iloc[0]

df_h2_burden_ssp
 


,10CH4_SSP119,10CH4_SSP434,10CH4_SSP585,CNTR_SSP119,CNTR_SSP434,CNTR_SSP585,antro10_SSP119,antro10_SSP434,antro10_SSP585
Scenario,,,,,,,,,
YR6,188,235,245,181,225,234,202,247,256
deltaH2,6.99,9.73,10.8,0.00,0.00,0.00,21.5,22.0,21.6


### 1.2 H2 loss
Hydrogen loss happens through two main processes. The largest loss is through soil sink. Remaining hydrogen is lost through reactions with OH as it ascends through the atmosphere. 

#### H2 dry deposition [Tg/yr]

In [7]:
file = 'H2_drydep.txt'
df_h2_drydep_conc = pd.read_csv(path + file, sep=';',index_col=0,header=0)
df_h2_drydep_conc = df_h2_drydep_conc['OSLOCTM3']
df_h2_drydep_conc

Scenario
CTRL    59.5
10H2    0.00
10CH4   59.5
Name: OSLOCTM3, dtype: float64

In [8]:
file = 'OSLOCTM3-emi_prod-loss-H2_all_YR13.csv'
df_h2_prodloss = pd.read_csv(path + file, sep=',',index_col=0,header=0)
df_h2_drydep = df_h2_prodloss.loc['Drydeposition']
df_h2_drydep

CNTR       58.0
antro01    58.1
antro1     58.7
antro10    65.0
antro100    128
nemo       58.7
epia       58.7
munich     58.7
usdrydep   58.8
maud       58.7
zep        58.7
maxdep     58.7
Name: Drydeposition, dtype: float64

In [9]:
file = 'OSLOCTM3-emi_prod-loss-H2_all_YR6.csv'
df_h2_prodloss_ssp = pd.read_csv(pathssp + file, sep=',',index_col=0,header=0)
df_h2_drydep_ssp = df_h2_prodloss_ssp.loc['Drydeposition']
df_h2_drydep_ssp

10CH4_SSP119     53.4
10CH4_SSP434     66.1
10CH4_SSP585     68.9
CNTR_SSP119      51.5
CNTR_SSP434      63.5
CNTR_SSP585      65.9
antro10_SSP119   58.5
antro10_SSP434   70.6
antro10_SSP585   72.9
Name: Drydeposition, dtype: float64

#### H2 atmospheric loss [Tg/yr]

In [10]:
file = 'H2_atm_loss.txt'
df_h2_atmloss_conc = pd.read_csv(path + file, sep=';',index_col=0,header=0)
df_h2_atmloss_conc = df_h2_atmloss_conc['OSLOCTM3']
df_h2_atmloss_conc


Scenario
CTRL    27.9
10H2    0.00
10CH4   27.2
Name: OSLOCTM3, dtype: float64

In [11]:
df_h2_atmloss = df_h2_prodloss.loc['Total loss'] - df_h2_prodloss.loc['Drydeposition']
df_h2_atmloss

CNTR       29.2
antro01    29.2
antro1     29.5
antro10    32.1
antro100   58.0
nemo       29.5
epia       29.4
munich     29.5
usdrydep   29.4
maud       29.5
zep        29.4
maxdep     29.4
dtype: float64

In [12]:
df_h2_atmloss_ssp = df_h2_prodloss_ssp.loc['Total loss'] - df_h2_prodloss_ssp.loc['Drydeposition']
df_h2_atmloss_ssp

10CH4_SSP119     25.1
10CH4_SSP434     28.6
10CH4_SSP585     31.4
CNTR_SSP119      24.9
CNTR_SSP434      28.3
CNTR_SSP585      31.0
antro10_SSP119   27.8
antro10_SSP434   31.0
antro10_SSP585   33.8
dtype: float64

#### H2 total loss [Tg/yr]:

In [13]:
df_h2_loss_conc = df_h2_atmloss_conc + df_h2_drydep_conc
df_h2_loss_conc


Scenario
CTRL    87.4
10H2    0.00
10CH4   86.7
Name: OSLOCTM3, dtype: float64

In [14]:
df_h2_loss = df_h2_atmloss+ df_h2_drydep
df_h2_loss


CNTR       87.2
antro01    87.3
antro1     88.2
antro10    97.1
antro100    186
nemo       88.2
epia       88.2
munich     88.2
usdrydep   88.2
maud       88.2
zep        88.2
maxdep     88.2
dtype: float64

In [15]:
df_h2_loss_ssp = df_h2_atmloss_ssp+ df_h2_drydep_ssp
df_h2_loss_ssp

10CH4_SSP119     78.5
10CH4_SSP434     94.7
10CH4_SSP585      100
CNTR_SSP119      76.5
CNTR_SSP434      91.8
CNTR_SSP585      97.0
antro10_SSP119   86.2
antro10_SSP434    102
antro10_SSP585    107
dtype: float64

### 1.3 H2 production

#### H2 atm. prod [Tg/yr]

In [16]:
file = 'H2_atm_prod.txt'
df_h2_atmprod_conc = pd.read_csv(path + file, sep=';',index_col=0,header=0)
df_h2_atmprod_conc = df_h2_atmprod_conc['OSLOCTM3']
df_h2_atmprod_conc

Scenario
CTRL    55.9
10H2    0.00
10CH4   58.7
Name: OSLOCTM3, dtype: float64

In [17]:
file = 'OSLOCTM3-emi_emissions_H2.csv'
df_h2_emis = pd.read_csv(path + file, sep=',',index_col=0,header=0)
df_h2_emis

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Total emission [Tg],32.2,32.3,33.2,42.3,132,33.2,33.2,33.2,33.2,33.2,33.2,33.2


In [18]:
file = 'OSLOCTM3-emi_emissions_H2.csv'
df_h2_emis_ssp = pd.read_csv(pathssp + file, sep=',',index_col=0,header=0)
df_h2_emis_ssp

,10CH4_SSP119,10CH4_SSP434,10CH4_SSP585,CNTR_SSP119,CNTR_SSP434,CNTR_SSP585,antro10_SSP119,antro10_SSP434,antro10_SSP585
Total emission [Tg],32.2,32.2,32.2,32.2,32.2,32.2,42.3,42.3,42.3


In [19]:
df_h2_atmprod = df_h2_prodloss.loc['Total production'] - df_h2_emis.iloc[0]
df_h2_atmprod

CNTR       55.8
antro01    55.8
antro1     55.8
antro10    55.7
antro100   54.9
nemo       55.8
epia       55.8
munich     55.8
usdrydep   55.8
maud       55.8
zep        55.8
maxdep     55.8
dtype: float64

In [20]:
df_h2_atmprod_ssp = df_h2_prodloss_ssp.loc['Total production'] - df_h2_emis_ssp.iloc[0]
df_h2_atmprod_ssp

10CH4_SSP119     47.2
10CH4_SSP434     63.4
10CH4_SSP585     69.0
CNTR_SSP119      45.0
CNTR_SSP434      60.4
CNTR_SSP585      65.6
antro10_SSP119   44.9
antro10_SSP434   60.3
antro10_SSP585   65.5
dtype: float64

### 1.4 Estimated H2 emissions (Total loss = Total prod + emission)

In the runs with fixed surface boundary conditions for H2, there are two unknowns; emissions and soil sink. The models do calculate the soil sink based on their own dry deposition schemes, but it does not influence the H2 in the atmosphere.  
Emission driven runs, are driven by emission estimates and use the dry deposition scheme to calculate the concentration at the surface. At steady state, the total production and total loss must balance:

In [21]:
df_h2_estemis_conc = df_h2_atmloss_conc + df_h2_drydep_conc - df_h2_atmprod_conc
df_h2_estemis_conc

Scenario
CTRL    31.5
10H2    0.00
10CH4   28.0
Name: OSLOCTM3, dtype: float64

In [22]:
df_h2_estemis_conc.loc['10H2']-df_h2_estemis_conc.loc['CTRL']

-31.526913320626726

In [23]:
df_h2_estemis = df_h2_atmloss + df_h2_drydep - df_h2_atmprod
df_h2_estemis

CNTR       31.4
antro01    31.5
antro1     32.4
antro10    41.4
antro100    131
nemo       32.4
epia       32.4
munich     32.4
usdrydep   32.4
maud       32.4
zep        32.4
maxdep     32.4
dtype: float64

In [24]:
df_h2_emis

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Total emission [Tg],32.2,32.3,33.2,42.3,132,33.2,33.2,33.2,33.2,33.2,33.2,33.2


In [25]:
#With more years these should be more similar..

In [26]:
df_h2_estemis_ssp = df_h2_atmloss_ssp + df_h2_drydep_ssp - df_h2_atmprod_ssp
df_h2_estemis_ssp

10CH4_SSP119     31.4
10CH4_SSP434     31.2
10CH4_SSP585     31.2
CNTR_SSP119      31.4
CNTR_SSP434      31.3
CNTR_SSP585      31.3
antro10_SSP119   41.3
antro10_SSP434   41.2
antro10_SSP585   41.2
dtype: float64

For the emission driven runs, the emissions calculated based on loss terms are similar to the emissions used in these runs.

### 1.5 H2 surface concentration [ppb]

In [27]:
file = 'H2_surfconc.txt'
df_h2_surfconc_conc = pd.read_csv(path + file, sep=';',index_col=0,header=0)
delta = df_h2_surfconc_conc.loc['10H2']-df_h2_surfconc_conc.loc['CTRL']
delta.name = 'deltaH2'
df_h2_surfconc_conc = df_h2_surfconc_conc.append(delta)
df_h2_surfconc_conc = df_h2_surfconc_conc['OSLOCTM3']
df_h2_surfconc_conc

Scenario
CTRL      532
10H2      nan
deltaH2   nan
Name: OSLOCTM3, dtype: float64

In [28]:
file = 'OSLOCTM3-emi_surfconc-H2.csv'
df_h2_surfconc = pd.read_csv(path + file, sep=',',index_col=0,header=0)
df_h2_surfconc.loc['delta'] = df_h2_surfconc.iloc[0]-df_h2_surfconc['CNTR'].iloc[0]
df_h2_surfconc
#delta = df_h2_surfconc_conc.loc['10H2']-df_h2_surfconc_conc.loc['CTRL']

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
YR13,559,560,565,621,"1,189",567,565,565,564,568,566,565
delta,0.00,0.62,6.22,62.2,630,7.85,6.01,6.23,5.15,9.15,6.63,5.42


In [29]:
file = 'OSLOCTM3-emi_surfconc-H2.csv'
df_h2_surfconc_ssp = pd.read_csv(pathssp + file, sep=',',index_col=0,header=0)

ssps = ['SSP119','SSP434','SSP585']
df_h2_surfconc_ssp.loc['delta'] = np.nan
df_h2_surfconc_ssp
for ssp in ssps:
    name = '10CH4_'+ssp
    df_h2_surfconc_ssp[name].loc['delta'] = df_h2_surfconc_ssp[name].iloc[0]-df_h2_surfconc_ssp['CNTR_'+ssp].iloc[0]
    name = 'antro10_'+ssp
    df_h2_surfconc_ssp[name].loc['delta'] = df_h2_surfconc_ssp[name].iloc[0]-df_h2_surfconc_ssp['CNTR_'+ssp].iloc[0]
    name = 'CNTR_'+ssp
    df_h2_surfconc_ssp[name].loc['delta'] = df_h2_surfconc_ssp[name].iloc[0]-df_h2_surfconc_ssp['CNTR_'+ssp].iloc[0]

df_h2_surfconc_ssp

,10CH4_SSP119,10CH4_SSP434,10CH4_SSP585,CNTR_SSP119,CNTR_SSP434,CNTR_SSP585,antro10_SSP119,antro10_SSP434,antro10_SSP585
YR6,514,639,667,495,613,638,558,677,701
delta,18.7,26.0,29.0,0.00,0.00,0.00,62.3,63.7,62.7


### 1.6 H2 lifetime [yr]
We calculate the lifetime as burden divided by loss.

#### H2 total lifetime [yr]

In [30]:
df_h2_lifetime_conc = df_h2_burden_conc/df_h2_loss_conc
df_h2_lifetime_conc


Scenario
CTRL    2.23
10H2     nan
10CH4   2.26
Name: OSLOCTM3, dtype: float64

In [31]:
df_h2_lifetime = df_h2_burden.iloc[0]/df_h2_loss
df_h2_lifetime

CNTR       2.35
antro01    2.35
antro1     2.34
antro10    2.33
antro100   2.27
nemo       2.35
epia       2.34
munich     2.34
usdrydep   2.34
maud       2.35
zep        2.34
maxdep     2.34
dtype: float64

In [32]:
df_h2_lifetime_ssp = df_h2_burden_ssp.iloc[0]/df_h2_loss_ssp
df_h2_lifetime_ssp

10CH4_SSP119     2.39
10CH4_SSP434     2.48
10CH4_SSP585     2.44
CNTR_SSP119      2.37
CNTR_SSP434      2.45
CNTR_SSP585      2.41
antro10_SSP119   2.35
antro10_SSP434   2.43
antro10_SSP585   2.40
dtype: float64

#### H2 atmospheric lifetime [yr]
The atmospheric lifetime is the burden divided by only the atmospheric loss. This is the lifetime of the fraction of hydrogen which is not dry deposited.

In [33]:
df_h2_atm_lifetime_conc = df_h2_burden_conc/df_h2_atmloss_conc
df_h2_atm_lifetime_conc

Scenario
CTRL    6.99
10H2     nan
10CH4   7.20
Name: OSLOCTM3, dtype: float64

In [34]:
df_h2_atm_lifetime = df_h2_burden.iloc[0]/df_h2_atmloss
df_h2_atm_lifetime

CNTR       7.02
antro01    7.02
antro1     7.02
antro10    7.04
antro100   7.28
nemo       7.02
epia       7.02
munich     7.02
usdrydep   7.02
maud       7.02
zep        7.02
maxdep     7.02
dtype: float64

In [35]:
df_h2_atm_lifetime_ssp = df_h2_burden_ssp.iloc[0]/df_h2_atmloss_ssp
df_h2_atm_lifetime_ssp

10CH4_SSP119     7.48
10CH4_SSP434     8.21
10CH4_SSP585     7.81
CNTR_SSP119      7.26
CNTR_SSP434      7.95
CNTR_SSP585      7.55
antro10_SSP119   7.29
antro10_SSP434   7.97
antro10_SSP585   7.57
dtype: float64

#### H2 soil sink lifetime [yr]

In [36]:
df_h2_soil_sink_lifetime_conc = df_h2_burden_conc/df_h2_drydep_conc
df_h2_soil_sink_lifetime_conc

Scenario
CTRL    3.28
10H2     nan
10CH4   3.29
Name: OSLOCTM3, dtype: float64

In [37]:
df_h2_soil_sink_lifetime = df_h2_burden.iloc[0]/df_h2_drydep
df_h2_soil_sink_lifetime

CNTR       3.53
antro01    3.53
antro1     3.52
antro10    3.48
antro100   3.30
nemo       3.53
epia       3.52
munich     3.52
usdrydep   3.51
maud       3.53
zep        3.52
maxdep     3.52
dtype: float64

In [38]:
df_h2_soil_sink_lifetime_ssp = df_h2_burden_ssp.iloc[0]/df_h2_drydep_ssp
df_h2_soil_sink_lifetime_ssp

10CH4_SSP119     3.52
10CH4_SSP434     3.55
10CH4_SSP585     3.56
CNTR_SSP119      3.51
CNTR_SSP434      3.54
CNTR_SSP585      3.55
antro10_SSP119   3.46
antro10_SSP434   3.50
antro10_SSP585   3.51
dtype: float64

### 1.7 H2 flux  [Tg/yr]

The hydrogen flux is calculated as the burden divided by the total hydrogen lifetime. 

The hydrogen lifetime is calculated as burden divided by the total loss (soil sink + atm.loss). 

In stedy state, this is equal to the total loss (se check below)

The difference in calculated flux in the perturbed and control run is calculated. These numbers include feedbacks. 

For the GWP calculations, the radiative forcing in the steady state simulations are divided by these flux numbers.

In [39]:
df_h2_flux_conc = df_h2_burden_conc/df_h2_lifetime_conc
#Add delta flux 10H2:
delta = df_h2_flux_conc.loc['10H2']-df_h2_flux_conc.loc['CTRL']
#delta.name = 'deltaH2'
#df_h2_flux_conc = df_h2_flux_conc.append(delta)
df_h2_flux_conc['deltaH2'] = delta

#Add delta flux 10CH4:
delta = df_h2_flux_conc.loc['10CH4']-df_h2_flux_conc.loc['CTRL']
#delta.name = 'deltaCH4'
#df_h2_flux_conc = df_h2_flux_conc.append(delta)
df_h2_flux_conc['deltaCH4'] = delta
df_h2_flux_conc


Scenario
CTRL        87.4
10H2         nan
10CH4       86.7
deltaH2      nan
deltaCH4   -0.75
Name: OSLOCTM3, dtype: float64

In [40]:
df_h2_flux = df_h2_burden.iloc[0]/df_h2_lifetime
df_h2_flux.name = 'h2_flux'

delta = df_h2_flux-df_h2_flux['CNTR']
delta.name = 'deltaH2'

df_h2_flux =pd.concat([df_h2_flux,delta],axis=1)

df_h2_flux['deltaCH4'] = df_h2_flux_conc['deltaCH4']
df_h2_flux
##Add delta flux 10CH4:
#delta = df_h2_flux.loc['10CH4']-df_h2_flux.loc['CTRL']
##delta.name = 'deltaCH4'
##df_h2_flux_conc = df_h2_flux_conc.append(delta)
#df_h2_flux_conc['deltaCH4'] = delta
#df_h2_flux_conc

,h2_flux,deltaH2,deltaCH4
CNTR,87.2,0.00,-0.75
antro01,87.3,0.10,-0.75
antro1,88.2,0.99,-0.75
antro10,97.1,9.90,-0.75
antro100,186,99.1,-0.75
nemo,88.2,0.99,-0.75
epia,88.2,0.99,-0.75
munich,88.2,0.99,-0.75
usdrydep,88.2,0.99,-0.75
maud,88.2,0.99,-0.75


In [41]:
df_h2_flux_ssp = df_h2_burden_ssp.iloc[0]/df_h2_lifetime_ssp
df_h2_flux_ssp.name = 'h2_flux'

df_h2_flux_ssp = df_h2_flux_ssp.to_frame()

df_h2_flux_ssp['deltaH2'] = np.nan
df_h2_flux_ssp['deltaCH4'] = np.nan

ssps = ['SSP119','SSP434','SSP585']

for ssp in ssps:
    name = '10CH4_'+ssp
    df_h2_flux_ssp['deltaH2'].loc[name] = df_h2_flux_ssp['h2_flux'].loc[name]-df_h2_flux_ssp['h2_flux'].loc['CNTR_'+ssp]
    name = 'antro10_'+ssp
    df_h2_flux_ssp['deltaH2'].loc[name] = df_h2_flux_ssp['h2_flux'].loc[name]-df_h2_flux_ssp['h2_flux'].loc['CNTR_'+ssp]
    name = 'CNTR_'+ssp
    df_h2_flux_ssp['deltaH2'].loc[name] = df_h2_flux_ssp['h2_flux'].loc[name]-df_h2_flux_ssp['h2_flux'].loc['CNTR_'+ssp]

    
#delta = df_h2_flux-df_h2_flux['CNTR']
#delta.name = 'deltaH2'
#
#df_h2_flux =pd.concat([df_h2_flux,delta],axis=1)
#
#df_h2_flux['deltaCH4'] = df_h2_flux_conc['deltaCH4']
#df_h2_flux
###Add delta flux 10CH4:
##delta = df_h2_flux.loc['10CH4']-df_h2_flux.loc['CTRL']
###delta.name = 'deltaCH4'
###df_h2_flux_conc = df_h2_flux_conc.append(delta)
##df_h2_flux_conc['deltaCH4'] = delta
df_h2_flux_ssp

,h2_flux,deltaH2,deltaCH4
10CH4_SSP119,78.5,2.08,nan
10CH4_SSP434,94.7,2.91,nan
10CH4_SSP585,100,3.28,nan
CNTR_SSP119,76.5,0.00,nan
CNTR_SSP434,91.8,0.00,nan
CNTR_SSP585,97.0,0.00,nan
antro10_SSP119,86.2,9.79,nan
antro10_SSP434,102,9.77,nan
antro10_SSP585,107,9.77,nan


In the methane run, the hydrogen surface concentration is kept fixed. Enhancing methane would influence H2. The hydrogen concentration would have increased, but since we run with fixed concentration, there is a negative flux to compensate. So the increased flux in H2 due to methane is -1*deltaCH4.

In [42]:
#Check
df_h2_loss_conc

Scenario
CTRL    87.4
10H2    0.00
10CH4   86.7
Name: OSLOCTM3, dtype: float64

In [43]:
df_h2_loss

CNTR       87.2
antro01    87.3
antro1     88.2
antro10    97.1
antro100    186
nemo       88.2
epia       88.2
munich     88.2
usdrydep   88.2
maud       88.2
zep        88.2
maxdep     88.2
dtype: float64

In [44]:
df_h2_loss_ssp

10CH4_SSP119     78.5
10CH4_SSP434     94.7
10CH4_SSP585      100
CNTR_SSP119      76.5
CNTR_SSP434      91.8
CNTR_SSP585      97.0
antro10_SSP119   86.2
antro10_SSP434    102
antro10_SSP585    107
dtype: float64

## 2. Methane results

### 2.1 CH4 burden [Tg]

In [45]:
file = 'CH4_burden.txt'
df_ch4_burden_conc = pd.read_csv(path + file, sep=';',index_col=0,header=0)
delta = df_ch4_burden_conc.loc['10CH4']-df_ch4_burden_conc.loc['CTRL']
delta.name = 'deltaCH4'
df_ch4_burden_conc = df_ch4_burden_conc.append(delta)
df_ch4_burden_conc = df_ch4_burden_conc['OSLOCTM3']
df_ch4_burden_conc

Scenario
CTRL       4,975
10H2         nan
10CH4      5,474
deltaCH4     498
Name: OSLOCTM3, dtype: float64

In [46]:
file = 'OSLOCTM3-emi_burden-CH4.csv'
df_ch4_burden = pd.read_csv(path + file, sep=',',index_col=0,header=0)
df_ch4_burden.loc['deltaCH4'] = df_ch4_burden.iloc[0]-df_ch4_burden['CNTR'].iloc[0]
df_ch4_burden

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
YR13,"4,975","4,975","4,975","4,975","4,975","4,975","4,975","4,975","4,975","4,975","4,975","4,975"
deltaCH4,0.00,0.00,0.00,0.05,0.43,0.01,0.00,0.00,0.00,0.01,0.00,0.00


In [47]:
file = 'OSLOCTM3-emi_burden-CH4.csv'
df_ch4_burden_ssp = pd.read_csv(pathssp + file, sep=',',index_col=0,header=0)
ssps = ['SSP119','SSP434','SSP585']
df_ch4_burden_ssp.loc['deltaCH4'] = np.nan
for ssp in ssps:
    name = '10CH4_'+ssp
    df_ch4_burden_ssp[name].loc['deltaCH4'] = df_ch4_burden_ssp[name].iloc[0]-df_ch4_burden_ssp['CNTR_'+ssp].iloc[0]
    name = 'antro10_'+ssp
    df_ch4_burden_ssp[name].loc['deltaCH4'] = df_ch4_burden_ssp[name].iloc[0]-df_ch4_burden_ssp['CNTR_'+ssp].iloc[0]
    name = 'CNTR_'+ssp
    df_ch4_burden_ssp[name].loc['deltaCH4'] = df_ch4_burden_ssp[name].iloc[0]-df_ch4_burden_ssp['CNTR_'+ssp].iloc[0]


df_ch4_burden_ssp

,10CH4_SSP119,10CH4_SSP434,10CH4_SSP585,CNTR_SSP119,CNTR_SSP434,CNTR_SSP585,antro10_SSP119,antro10_SSP434,antro10_SSP585
Scenario,,,,,,,,,
YR6,"4,309","6,715","7,385","3,916","6,104","6,713","3,916","6,104","6,713"
deltaCH4,392,611,672,0.00,0.00,0.00,0.09,0.06,0.06


### 2.2 CH4 atmospheric loss [Tg/yr]

This is atmospheric loss due to OH.

In [48]:
file = 'CH4_loss.txt'
df_ch4_loss_conc = pd.read_csv(path + file, sep=';',index_col=0,header=0)
df_ch4_loss_conc = df_ch4_loss_conc['OSLOCTM3']
df_ch4_loss_conc

Scenario
CTRL    674
10H2    nan
10CH4   719
Name: OSLOCTM3, dtype: float64

In [49]:
file = 'OSLOCTM3-emi_prod-loss-CH4_all_YR13.csv'
df_ch4_prodloss = pd.read_csv(path + file, sep=',',index_col=0,header=0)
df_ch4_loss=df_ch4_prodloss.loc['CH4 + OH -> H2O + CH3']
df_ch4_loss

CNTR       673
antro01    673
antro1     673
antro10    670
antro100   644
nemo       672
epia       673
munich     673
usdrydep   673
maud       672
zep        673
maxdep     673
Name: CH4 + OH -> H2O + CH3, dtype: float64

In [50]:
file = 'OSLOCTM3-emi_prod-loss-CH4_all_YR6.csv'
df_ch4_prodloss_ssp = pd.read_csv(pathssp + file, sep=',',index_col=0,header=0)
df_ch4_loss_ssp=df_ch4_prodloss_ssp.loc['CH4 + OH -> H2O + CH3']
df_ch4_loss_ssp

10CH4_SSP119     545
10CH4_SSP434     769
10CH4_SSP585     892
CNTR_SSP119      511
CNTR_SSP434      724
CNTR_SSP585      841
antro10_SSP119   508
antro10_SSP434   721
antro10_SSP585   837
Name: CH4 + OH -> H2O + CH3, dtype: float64

### 2.3 CH4 surface concentration [ppb]

In [51]:
file = 'CH4_surfconc.txt'
df_ch4_surfconc_conc = pd.read_csv(path + file, sep=';',index_col=0,header=0)
delta = df_ch4_surfconc_conc.loc['10CH4']-df_ch4_surfconc_conc.loc['CTRL']
delta.name = 'deltaCH4'
df_ch4_surfconc_conc = df_ch4_surfconc_conc.append(delta)
df_ch4_surfconc_conc = df_ch4_surfconc_conc['OSLOCTM3']
df_ch4_surfconc_conc

Scenario
CTRL       1,813
10CH4      1,994
deltaCH4     181
Name: OSLOCTM3, dtype: float64

In [52]:
file = 'OSLOCTM3-emi_surfconc-CH4.csv'
df_ch4_surfconc = pd.read_csv(path + file, sep=',',index_col=0,header=0)
df_ch4_surfconc.loc['delta'] = df_ch4_surfconc.iloc[0]-df_ch4_surfconc['CNTR'].iloc[0]
df_ch4_surfconc.loc['delta'] = df_ch4_surfconc_conc['deltaCH4']
df_ch4_surfconc

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
YR13,"1,813","1,813","1,813","1,813","1,813","1,813","1,813","1,813","1,813","1,813","1,813","1,813"
delta,181,181,181,181,181,181,181,181,181,181,181,181


In [53]:
file = 'OSLOCTM3-emi_surfconc-CH4.csv'
df_ch4_surfconc_ssp = pd.read_csv(pathssp + file, sep=',',index_col=0,header=0)

ssps = ['SSP119','SSP434','SSP585']
df_ch4_surfconc_ssp.loc['delta'] = np.nan
df_ch4_surfconc_ssp
for ssp in ssps:
    name = '10CH4_'+ssp
    df_ch4_surfconc_ssp[name].loc['delta'] = df_ch4_surfconc_ssp[name].iloc[0]-df_ch4_surfconc_ssp['CNTR_'+ssp].iloc[0]
    name = 'antro10_'+ssp
    df_ch4_surfconc_ssp[name].loc['delta'] = df_ch4_surfconc_ssp[name].iloc[0]-df_ch4_surfconc_ssp['CNTR_'+ssp].iloc[0]
    name = 'CNTR_'+ssp
    df_ch4_surfconc_ssp[name].loc['delta'] = df_ch4_surfconc_ssp[name].iloc[0]-df_ch4_surfconc_ssp['CNTR_'+ssp].iloc[0]

df_ch4_surfconc_ssp

,10CH4_SSP119,10CH4_SSP434,10CH4_SSP585,CNTR_SSP119,CNTR_SSP434,CNTR_SSP585,antro10_SSP119,antro10_SSP434,antro10_SSP585
YR6,"1,570","2,445","2,690","1,427","2,223","2,446","1,427","2,223","2,446"
delta,143,222,245,0.00,0.00,0.00,0.00,0.00,0.00


### 2.4 CH4 lifetime [yr]

#### Lifetime due to OH [yr]

In [54]:
df_ch4_lifetime_conc = df_ch4_burden_conc.drop('deltaCH4')/df_ch4_loss_conc
df_ch4_lifetime_conc

Scenario
CTRL    7.38
10H2     nan
10CH4   7.62
Name: OSLOCTM3, dtype: float64

In [55]:
df_ch4_lifetime = df_ch4_burden.drop('deltaCH4')/df_ch4_loss
df_ch4_lifetime

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
YR13,7.39,7.39,7.40,7.43,7.73,7.40,7.40,7.40,7.40,7.40,7.40,7.40


In [56]:
df_ch4_lifetime_ssp = df_ch4_burden_ssp.drop('deltaCH4')/df_ch4_loss_ssp
df_ch4_lifetime_ssp

,10CH4_SSP119,10CH4_SSP434,10CH4_SSP585,CNTR_SSP119,CNTR_SSP434,CNTR_SSP585,antro10_SSP119,antro10_SSP434,antro10_SSP585
Scenario,,,,,,,,,
YR6,7.91,8.73,8.28,7.67,8.43,7.99,7.71,8.47,8.02


#### Total CH4 lifetime [yr]

Inverse lifetimes (mean loss frequencies) are additive [Forster et al.,2007; Prather, 2007]. Added lifetime due to stratospheric chemistry in addion to OH and soil sink.

In [57]:
df_ch4_tot_lifetime_conc = 1.0/(1.0/df_ch4_lifetime_conc + 1.0/tau_strat + 1.0/tau_soil)
df_ch4_tot_lifetime_conc

Scenario
CTRL    6.85
10H2     nan
10CH4   7.06
Name: OSLOCTM3, dtype: float64

In [58]:
df_ch4_tot_lifetime = 1.0/(1.0/df_ch4_lifetime + 1.0/tau_strat + 1.0/tau_soil)
df_ch4_tot_lifetime

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
YR13,6.87,6.87,6.87,6.89,7.15,6.87,6.87,6.87,6.87,6.87,6.87,6.87


In [59]:
df_ch4_tot_lifetime_ssp = 1.0/(1.0/df_ch4_lifetime_ssp + 1.0/tau_strat + 1.0/tau_soil)
df_ch4_tot_lifetime_ssp

,10CH4_SSP119,10CH4_SSP434,10CH4_SSP585,CNTR_SSP119,CNTR_SSP434,CNTR_SSP585,antro10_SSP119,antro10_SSP434,antro10_SSP585
Scenario,,,,,,,,,
YR6,7.31,8.00,7.62,7.10,7.75,7.37,7.14,7.78,7.40


#### Other methane loss term, dervided based on the lifetime and burden.

In [60]:
#Soil loss:
df_ch4_loss_soil_conc = df_ch4_burden_conc.drop('deltaCH4')/tau_soil
df_ch4_loss_soil_conc

Scenario
CTRL    31.1
10H2     nan
10CH4   34.2
Name: OSLOCTM3, dtype: float64

In [61]:
df_ch4_loss_soil = df_ch4_burden.drop('deltaCH4')/tau_soil
df_ch4_loss_soil

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
YR13,31.1,31.1,31.1,31.1,31.1,31.1,31.1,31.1,31.1,31.1,31.1,31.1


In [62]:
df_ch4_loss_soil_ssp = df_ch4_burden_ssp.drop('deltaCH4')/tau_soil
df_ch4_loss_soil_ssp

,10CH4_SSP119,10CH4_SSP434,10CH4_SSP585,CNTR_SSP119,CNTR_SSP434,CNTR_SSP585,antro10_SSP119,antro10_SSP434,antro10_SSP585
Scenario,,,,,,,,,
YR6,26.9,42.0,46.2,24.5,38.2,42.0,24.5,38.2,42.0


In [63]:
#Strat chemistry loss (other than OH)
df_ch4_loss_other_strat_conc = df_ch4_burden_conc.drop('deltaCH4')/(tau_strat)
df_ch4_loss_other_strat_conc

Scenario
CTRL    20.7
10H2     nan
10CH4   22.8
Name: OSLOCTM3, dtype: float64

In [64]:
df_ch4_loss_other_strat = df_ch4_burden.drop('deltaCH4')/(tau_strat)
df_ch4_loss_other_strat

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
YR13,20.7,20.7,20.7,20.7,20.7,20.7,20.7,20.7,20.7,20.7,20.7,20.7


In [65]:
df_ch4_loss_other_strat_ssp = df_ch4_burden_ssp.drop('deltaCH4')/(tau_strat)
df_ch4_loss_other_strat_ssp

,10CH4_SSP119,10CH4_SSP434,10CH4_SSP585,CNTR_SSP119,CNTR_SSP434,CNTR_SSP585,antro10_SSP119,antro10_SSP434,antro10_SSP585
Scenario,,,,,,,,,
YR6,18.0,28.0,30.8,16.3,25.4,28.0,16.3,25.4,28.0


### 2.5 CH4 flux  [Tg/yr]

The methane flux is calculated as the burden divided by the total methane lifetime.

The difference in calculated flux in the perturbed and control run is calculated. These numbers include feedbacks.

In [66]:
df_ch4_flux_conc = df_ch4_burden_conc.drop('deltaCH4')/df_ch4_tot_lifetime_conc
##Add delta CH4 flux 10H2
#delta = df_ch4_flux_conc.loc['10H2']-df_ch4_flux_conc.loc['CTRL']
#delta.name = 'deltaH2'
#df_ch4_flux_conc = df_ch4_flux_conc.append(delta)
#Add delta CH4 flux 10CH4

#delta = df_ch4_flux_conc.loc['10CH4']-df_ch4_flux_conc.loc['CTRL']
#delta.name = 'deltaCH4'

#df_ch4_flux_conc = df_ch4_flux_conc.append(delta)
df_ch4_flux_conc['deltaCH4'] = df_ch4_flux_conc.loc['10CH4']-df_ch4_flux_conc.loc['CTRL']
df_ch4_flux_conc

##Add same flux in OsloCTM3-emi as in OsloCTM3
#df_ch4_flux['OSLOCTM3-emi'].loc['deltaCH4']= df_ch4_flux['OSLOCTM3'].loc['deltaCH4']
#df_ch4_flux


Scenario
CTRL        726
10H2        nan
10CH4       776
deltaCH4   49.7
Name: OSLOCTM3, dtype: float64

In [67]:
df_ch4_flux = df_ch4_burden.drop('deltaCH4')/df_ch4_tot_lifetime
df_ch4_flux.loc['deltaH2'] = df_ch4_flux.iloc[0]-df_ch4_flux['CNTR'].iloc[0]

#Add same ch4 flux in OsloCTM3 sensitivity tests as in OsloCTM3 conc
df_ch4_flux.loc['deltaCH4'] =  df_ch4_flux_conc['deltaCH4']
df_ch4_flux

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
YR13,725,725,724,722,696,724,724,724,724,724,724,724
deltaH2,0.00,-0.03,-0.30,-3.03,-28.7,-0.35,-0.27,-0.29,-0.25,-0.35,-0.29,-0.28
deltaCH4,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7


In [68]:
df_ch4_flux_ssp = df_ch4_burden_ssp.drop('deltaCH4')/df_ch4_tot_lifetime_ssp


#df_ch4_flux_ssp = df_ch4_flux_ssp.to_frame()

df_ch4_flux_ssp.loc['deltaH2'] = np.nan
df_ch4_flux_ssp.loc['deltaCH4'] = np.nan


ssps = ['SSP119','SSP434','SSP585']

for ssp in ssps:
    name = '10CH4_'+ssp
    df_ch4_flux_ssp.loc['deltaCH4'][name] = df_ch4_flux_ssp.iloc[0][name]-df_ch4_flux_ssp.iloc[0]['CNTR_'+ssp]
    name = 'antro10_'+ssp
    df_ch4_flux_ssp.loc['deltaCH4'][name] = df_ch4_flux_ssp.iloc[0][name]-df_ch4_flux_ssp.iloc[0]['CNTR_'+ssp]
    name = 'CNTR_'+ssp
    df_ch4_flux_ssp.loc['deltaCH4'][name] = df_ch4_flux_ssp.iloc[0][name]-df_ch4_flux_ssp.iloc[0]['CNTR_'+ssp]


df_ch4_flux_ssp

,10CH4_SSP119,10CH4_SSP434,10CH4_SSP585,CNTR_SSP119,CNTR_SSP434,CNTR_SSP585,antro10_SSP119,antro10_SSP434,antro10_SSP585
Scenario,,,,,,,,,
YR6,589,839,969,552,787,910,549,784,907
deltaH2,nan,nan,nan,nan,nan,nan,nan,nan,nan
deltaCH4,37.8,52.1,58.6,0.00,0.00,0.00,-2.90,-2.90,-3.13


## 3. Ozone burden and RF

### 3.1 Change in tropospheric ozone (DU)

The tropopause definition is the model layer in the control simulation where 150 ppbv ozone are reached. 

Keep in mind that:
for GFDL-emi 10H2 is H2 perturbation (200 Tg/yr) and 10CH4 is H2+CH4 pert. And that for OsloCTM-emi, the same sensitivity for methane perturbation is used as for OsloCTM concentration driven.

In [69]:
file = 'ozone_du_trop.txt'
df_ozone_du_trop_conc = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_ozone_du_trop_conc = df_ozone_du_trop_conc['OSLOCTM3']
df_ozone_du_trop_conc

Scenario
10H2     nan
10CH4   0.83
Name: OSLOCTM3, dtype: float64

In [70]:
#ADD here for senstest

In [71]:
#For OsloCTM-emi, use the methane results from the concentration driven methane experiment.
#df_ozone_du_trop['OSLOCTM3-emi'].loc['10CH4'] = df_ozone_du_trop['OSLOCTM3'].loc['10CH4'] 
#df_ozone_du_trop

### 3.2 Change in stratospheric ozone (DU)

In [72]:
file = 'ozone_du_strat.txt'
df_ozone_du_strat_conc = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_ozone_du_strat_conc = df_ozone_du_strat_conc['OSLOCTM3']
df_ozone_du_strat_conc

Scenario
10H2     nan
10CH4   0.84
Name: OSLOCTM3, dtype: float64

In [73]:
#ADD here for senstest


In [74]:
#For OsloCTM-emi, use the methane results from the concentration driven methane experiment.
#df_ozone_du_strat['OSLOCTM3-emi'].loc['10CH4'] = df_ozone_du_strat['OSLOCTM3'].loc['10CH4'] 
#df_ozone_du_strat

### 3.5 Ozone RF

Ozone RF is calculated using a radiative kernel (Skeie et al 2020) and the modelled changes in ozone. 

Keep in mind that:
for GFDL-emi 10H2 RF is forcing calculated in the H2 perturbation and 10CH4 the forcing calculated by the H2+CH4.

In [75]:
file = 'ozone_rf.txt'
df_ozone_rf_conc = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_ozone_rf_conc = df_ozone_rf_conc['OSLOCTM3'] 
df_ozone_rf_conc 

Scenario
10H2     nan
10CH4   40.8
Name: OSLOCTM3, dtype: float64

In [76]:
#ADD here for senstest
df_ozone_rf = pd.DataFrame(data=[],columns=df_ch4_flux.columns,index=df_ozone_rf_conc.index)
df_ozone_rf

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
10H2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10CH4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
file = 'RF_NRFmethod_OsloCTM3_senstest_yr26_net_yearly.csv'
ozone_rf_orig = pd.read_csv(path+file,sep=';',index_col=0,header=0,skiprows=3)
ozone_rf_orig['NET adj.']


CNTR (mW m-2)             nan
antro01-CNTR (mW m-2)    0.02
antro1-CNTR (mW m-2)     0.21
antro10-CNTR (mW m-2)    2.14
antro100-CNTR (mW m-2)   21.4
epia-CNTR (mW m-2)       0.18
nemo-CNTR (mW m-2)       0.23
munich-CNTR (mW m-2)     0.20
usdrydep-CNTR (mW m-2)   0.17
zep-CNTR (mW m-2)        0.19
maud-CNTR (mW m-2)       0.23
maxdep-CNTR (mW m-2)     0.20
Name: NET adj., dtype: float64

In [78]:
for scen in df_ozone_rf.columns[1:]:
    df_ozone_rf.loc['10H2'][scen] = ozone_rf_orig['NET adj.'].loc[scen+'-CNTR (mW m-2)']
df_ozone_rf
    


,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
10H2,NaN,0.02,0.21,2.14,21.4,0.23,0.18,0.20,0.17,0.23,0.19,0.20
10CH4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
#In OsloCTM-emi use the same RF in the methane pertubation as in OsloCTM3
df_ozone_rf.loc['10CH4'] = df_ozone_rf_conc.loc['10CH4'] 
df_ozone_rf

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
10H2,NaN,0.02,0.21,2.14,21.4,0.23,0.18,0.20,0.17,0.23,0.19,0.20
10CH4,40.8,40.8,40.8,40.8,40.8,40.8,40.8,40.8,40.8,40.8,40.8,40.8


Read the ozone RF from SSP sensitivity tests

In [80]:
ssps = ['SSP119','SSP434','SSP585']

df_ozone_rf_ssp = pd.DataFrame(data=[],columns=ssps,index=['pertH2','10CH4'])

for ssp in ssps:
    file = 'RF_NRFmethod_OsloCTM3_ssp_senstest_yr12_'+ssp+'_net_yearly.csv'
    ozone_rf_orig = pd.read_csv(pathssp+file,sep=';',index_col=0,header=0,skiprows=3)
    ozone_rf_orig['NET adj.']
    df_ozone_rf_ssp[ssp].loc['pertH2'] = ozone_rf_orig['NET adj.'].loc['antro10_'+ssp+'-CNTR_'+ssp+' (mW m-2)']
    df_ozone_rf_ssp[ssp].loc['10CH4'] = ozone_rf_orig['NET adj.'].loc['10CH4_'+ssp+'-CNTR_'+ssp+' (mW m-2)']
df_ozone_rf_ssp

,SSP119,SSP434,SSP585
pertH2,2.18,1.80,1.77
10CH4,33.5,43.9,49.7


## 4. Stratospheric H2O RF [mW m-2]

Stratospheric H2O RF calculated offline.

In [81]:
file = 'H2O_rf.txt'
df_h2o_rf_conc = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_h2o_rf_conc = df_h2o_rf_conc['OSLOCTM3']
df_h2o_rf_conc

Scenario
10H2     nan
10CH4   9.53
Name: OSLOCTM3, dtype: float64

In [82]:
#ADD here for senstest
df_h2o_rf = pd.DataFrame(data=[],columns=df_ch4_flux.columns,index=df_h2o_rf_conc.index)

In [83]:
file = 'strat_h2o_h2.csv'
df_h2o_rf_orig = pd.read_csv(path+file,sep='\t',index_col=0,header=0)
df_h2o_rf_orig

,an01,ant1,an10,a100,zepl,maud,nemo,epia,munc,usdd,maxd
NET,0.02,0.17,1.67,16.8,0.15,0.18,0.18,0.14,0.15,0.13,0.16


In [84]:
scen_dict_strat_h2o = {'an01':'antro01',
                       'ant1':'antro1',
                       'an10':'antro10',
                       'a100':'antro100',
                       'zepl':'zep',
                       'maud':'maud',
                       'nemo':'nemo',
                       'epia':'epia',
                       'munc':'munich',
                       'usdd':'usdrydep',
                       'maxd':'maxdep'}
for scen in scen_dict_strat_h2o:
    df_h2o_rf[scen_dict_strat_h2o[scen]].loc['10H2'] = df_h2o_rf_orig[scen].loc['NET']
df_h2o_rf

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
10H2,NaN,0.02,0.17,1.67,16.8,0.18,0.14,0.15,0.13,0.18,0.15,0.16
10CH4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
#In OsloCTM-emi use the same RF in the methane pertubation as in OsloCTM3
df_h2o_rf.loc['10CH4'] = df_h2o_rf_conc.loc['10CH4'] 
df_h2o_rf

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
10H2,NaN,0.02,0.17,1.67,16.8,0.18,0.14,0.15,0.13,0.18,0.15,0.16
10CH4,9.53,9.53,9.53,9.53,9.53,9.53,9.53,9.53,9.53,9.53,9.53,9.53


In [86]:
ssps = ['SSP119','SSP434','SSP585']

df_h2o_rf_ssp = pd.read_csv(pathssp+'strat_h2o_ssp.csv',sep=',',index_col=0,header=0) 
df_h2o_rf_ssp

,SSP119,SSP434,SSP585
pertH2,1.65,1.74,1.74
10CH4,10.9,17.1,18.9


## 5. Aerosol RF [mW m-2]

In [87]:
file = 'aerosol_rf.txt'
df_aerosol_rf_conc = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_aerosol_rf_conc = df_aerosol_rf_conc['OSLOCTM3']
df_aerosol_rf_conc

Scenario
10H2    nan
10CH4   nan
Name: OSLOCTM3, dtype: float64

In [88]:
#ADD here for senstest
df_aerosol_rf = pd.DataFrame(data=[],columns=df_ch4_flux.columns,index=df_h2o_rf_conc.index)
df_aerosol_rf

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
10H2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10CH4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
#In OsloCTM-emi use the same RF in the methane pertubation as in OsloCTM3
df_aerosol_rf.loc['10CH4'] = df_aerosol_rf_conc.loc['10CH4'] 
df_aerosol_rf

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
10H2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10CH4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



# Part II: GWP calculations:

The pulse integrated to infinity of the effects of a short lived climate forcer is equal to the change respones of its effects at steady state multiplied by the steady state lifetime of the short lived forcer(:cite:p:`Prather2002a` and :cite:p:`Prather2007a`). 

Prather 2002: prove that: (a) the steadystate pattern of impacts caused by specified emissions, multiplied by (b) the steady-state lifetime of the source gas for that emission pattern, is exactly equal to (c) the integral of all impacts - independent of the number and atmospheric residence times of secondary impacts. Therefore, the AGWP for hydrogen is identical whether calculating by integrating a pulse or by using the steady state changes per flux, given that the perturbation reaches steady state before 100 years. The longest-lived chemical mode here is keyed to methane, which has an e-folding lifetime of about 12 years. Our perturbed experiments are run to steate-state.


### Change in H2 surface conc. caused by 1 Tg H2/yr [ppb yr Tg-1]

This is not used for the GWP calculation. Only for the per flux table and for the feedback factor calulations.

In [90]:
df_ch4_flux

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
YR13,725,725,724,722,696,724,724,724,724,724,724,724
deltaH2,0.00,-0.03,-0.30,-3.03,-28.7,-0.35,-0.27,-0.29,-0.25,-0.35,-0.29,-0.28
deltaCH4,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7


In [91]:
df_ch4_flux_ssp

,10CH4_SSP119,10CH4_SSP434,10CH4_SSP585,CNTR_SSP119,CNTR_SSP434,CNTR_SSP585,antro10_SSP119,antro10_SSP434,antro10_SSP585
Scenario,,,,,,,,,
YR6,589,839,969,552,787,910,549,784,907
deltaH2,nan,nan,nan,nan,nan,nan,nan,nan,nan
deltaCH4,37.8,52.1,58.6,0.00,0.00,0.00,-2.90,-2.90,-3.13


In [92]:
df_surf_h2_per_h2_flux = df_h2_surfconc.loc['delta']/df_h2_flux['deltaH2']
df_surf_h2_per_h2_flux.name = 'surf_h2_per_h2_flux'
df_surf_h2_per_h2_flux

CNTR        nan
antro01    6.27
antro1     6.27
antro10    6.28
antro100   6.35
nemo       7.92
epia       6.05
munich     6.32
usdrydep   5.20
maud       9.24
zep        6.69
maxdep     5.48
Name: surf_h2_per_h2_flux, dtype: float64

In [93]:
df_h2_surfconc_ssp

,10CH4_SSP119,10CH4_SSP434,10CH4_SSP585,CNTR_SSP119,CNTR_SSP434,CNTR_SSP585,antro10_SSP119,antro10_SSP434,antro10_SSP585
YR6,514,639,667,495,613,638,558,677,701
delta,18.7,26.0,29.0,0.00,0.00,0.00,62.3,63.7,62.7


In [94]:
df_h2_flux_ssp

,h2_flux,deltaH2,deltaCH4
10CH4_SSP119,78.5,2.08,nan
10CH4_SSP434,94.7,2.91,nan
10CH4_SSP585,100,3.28,nan
CNTR_SSP119,76.5,0.00,nan
CNTR_SSP434,91.8,0.00,nan
CNTR_SSP585,97.0,0.00,nan
antro10_SSP119,86.2,9.79,nan
antro10_SSP434,102,9.77,nan
antro10_SSP585,107,9.77,nan


In [95]:
df_surf_h2_per_h2_flux_ssp = df_h2_surfconc_ssp.loc['delta']/df_h2_flux_ssp['deltaH2']
df_surf_h2_per_h2_flux_ssp.name = 'surf_h2_per_h2_flux'
df_surf_h2_per_h2_flux_ssp

10CH4_SSP119     9.01
10CH4_SSP434     8.94
10CH4_SSP585     8.84
CNTR_SSP119       nan
CNTR_SSP434       nan
CNTR_SSP585       nan
antro10_SSP119   6.36
antro10_SSP434   6.52
antro10_SSP585   6.42
Name: surf_h2_per_h2_flux, dtype: float64

### Change in CH4 flux caused by 1 TgH2 /yr (includes H2 feedback) [Tg CH4/Tg H2]:

The ch4_flux is multiplied by -1 (see above).

In [96]:
df_ch4_flux_per_h2_flux = -1.0*df_ch4_flux.loc['deltaH2']/df_h2_flux['deltaH2']
df_ch4_flux_per_h2_flux.name = 'ch4_flux_per_h2_flux'
df_ch4_flux_per_h2_flux

CNTR        nan
antro01    0.30
antro1     0.31
antro10    0.31
antro100   0.29
nemo       0.35
epia       0.28
munich     0.29
usdrydep   0.25
maud       0.35
zep        0.29
maxdep     0.28
Name: ch4_flux_per_h2_flux, dtype: float64

In [97]:
df_ch4_flux_ssp

,10CH4_SSP119,10CH4_SSP434,10CH4_SSP585,CNTR_SSP119,CNTR_SSP434,CNTR_SSP585,antro10_SSP119,antro10_SSP434,antro10_SSP585
Scenario,,,,,,,,,
YR6,589,839,969,552,787,910,549,784,907
deltaH2,nan,nan,nan,nan,nan,nan,nan,nan,nan
deltaCH4,37.8,52.1,58.6,0.00,0.00,0.00,-2.90,-2.90,-3.13


In [98]:
df_ch4_flux_ssp_v2 = pd.DataFrame(data=[],columns=['SSP119','SSP434','SSP585'],index=['deltaH2','deltaCH4'])

for scen in df_ch4_flux_ssp_v2:
        df_ch4_flux_ssp_v2[scen].loc['deltaH2'] = df_ch4_flux_ssp.loc['deltaCH4']['antro10_'+scen]
        df_ch4_flux_ssp_v2[scen].loc['deltaCH4'] = df_ch4_flux_ssp.loc['deltaCH4']['10CH4_'+scen]
df_ch4_flux_ssp_v2

,SSP119,SSP434,SSP585
deltaH2,-2.90,-2.90,-3.13
deltaCH4,37.8,52.1,58.6


In [99]:
df_h2_flux_ssp_v2 = pd.DataFrame(data=[],columns=['SSP119','SSP434','SSP585'],index=['deltaH2','deltaCH4'])

for scen in df_h2_flux_ssp_v2:
        df_h2_flux_ssp_v2[scen].loc['deltaH2'] = df_h2_flux_ssp['deltaH2'].loc['antro10_'+scen]
        df_h2_flux_ssp_v2[scen].loc['deltaCH4'] = df_h2_flux_ssp['deltaH2'].loc['10CH4_'+scen]
df_h2_flux_ssp_v2

,SSP119,SSP434,SSP585
deltaH2,9.79,9.77,9.77
deltaCH4,2.08,2.91,3.28


In [100]:
df_ch4_flux_per_h2_flux_ssp = -1.0*df_ch4_flux_ssp_v2.loc['deltaH2']/df_h2_flux_ssp_v2.loc['deltaH2']
df_ch4_flux_per_h2_flux_ssp.name = 'ch4_flux_per_h2_flux_ssp'
df_ch4_flux_per_h2_flux_ssp

SSP119   0.30
SSP434   0.30
SSP585   0.32
Name: ch4_flux_per_h2_flux_ssp, dtype: object

### Change in CH4 surface conc. caused by 1 Tg/yr CH4 [ppb yr/Tg CH4]

In [101]:
df_surf_ch4_per_ch4_flux_conc =  df_ch4_surfconc_conc.loc['deltaCH4']/df_ch4_flux_conc.loc['deltaCH4']
df_surf_ch4_per_ch4_flux_conc


3.650302696662327

In [102]:
df_surf_ch4_per_ch4_flux =  df_ch4_surfconc.loc['delta']/df_ch4_flux.loc['deltaCH4']
df_surf_ch4_per_ch4_flux.name = 'surf_ch4_per_ch4_flux'
df_surf_ch4_per_ch4_flux

CNTR       3.65
antro01    3.65
antro1     3.65
antro10    3.65
antro100   3.65
nemo       3.65
epia       3.65
munich     3.65
usdrydep   3.65
maud       3.65
zep        3.65
maxdep     3.65
Name: surf_ch4_per_ch4_flux, dtype: float64

OsloCTM3-emi set equal to OsloCTM3 concentration driven.

In [103]:
df_surf_ch4_per_ch4_flux[:] = df_surf_ch4_per_ch4_flux_conc
df_surf_ch4_per_ch4_flux

CNTR       3.65
antro01    3.65
antro1     3.65
antro10    3.65
antro100   3.65
nemo       3.65
epia       3.65
munich     3.65
usdrydep   3.65
maud       3.65
zep        3.65
maxdep     3.65
Name: surf_ch4_per_ch4_flux, dtype: float64

In [104]:
df_ch4_surfconc_ssp_v2 = pd.DataFrame(data=[],columns=['SSP119','SSP434','SSP585'],index=['delta'])
for scen in df_ch4_surfconc_ssp_v2.columns:
        df_ch4_surfconc_ssp_v2[scen].loc['delta'] = df_ch4_surfconc_ssp.loc['delta']['10CH4_'+scen]

df_ch4_surfconc_ssp_v2

,SSP119,SSP434,SSP585
delta,143,222,245


In [105]:
df_surf_ch4_per_ch4_flux_ssp =  df_ch4_surfconc_ssp_v2.loc['delta']/df_ch4_flux_ssp_v2.loc['deltaCH4']
df_surf_ch4_per_ch4_flux_ssp.name = 'surf_ch4_per_ch4_flux'
df_surf_ch4_per_ch4_flux_ssp

SSP119   3.78
SSP434   4.27
SSP585   4.17
Name: surf_ch4_per_ch4_flux, dtype: object

### Change in CH4 surface concentration per emission H2 [ppb yr /Tg H2]

In [106]:
df_surf_ch4_per_h2_flux = df_surf_ch4_per_ch4_flux*df_ch4_flux_per_h2_flux
df_surf_ch4_per_h2_flux.name = 'surf_ch4_per_h2_flux'
df_surf_ch4_per_h2_flux

CNTR        nan
antro01    1.11
antro1     1.12
antro10    1.12
antro100   1.06
nemo       1.29
epia       1.01
munich     1.08
usdrydep   0.92
maud       1.29
zep        1.06
maxdep     1.03
Name: surf_ch4_per_h2_flux, dtype: float64

In [107]:
df_surf_ch4_per_h2_flux_ssp = df_surf_ch4_per_ch4_flux_ssp*df_ch4_flux_per_h2_flux_ssp
df_surf_ch4_per_h2_flux_ssp.name = 'surf_ch4_per_h2_flux'
df_surf_ch4_per_h2_flux_ssp

SSP119   1.12
SSP434   1.27
SSP585   1.34
Name: surf_ch4_per_h2_flux, dtype: object

In [108]:
df_h2_flux

,h2_flux,deltaH2,deltaCH4
CNTR,87.2,0.00,-0.75
antro01,87.3,0.10,-0.75
antro1,88.2,0.99,-0.75
antro10,97.1,9.90,-0.75
antro100,186,99.1,-0.75
nemo,88.2,0.99,-0.75
epia,88.2,0.99,-0.75
munich,88.2,0.99,-0.75
usdrydep,88.2,0.99,-0.75
maud,88.2,0.99,-0.75


### Change in H2 flux caused by 1 TgCH4/yr [Tg H2/Tg CH4]

We multiply by -1 (see above)

In [109]:
df_h2_flux_per_ch4_flux = -1.0*df_h2_flux['deltaCH4']/df_ch4_flux.loc['deltaCH4']
df_h2_flux_per_ch4_flux.name = 'h2_flux_per_ch4_flux'
df_h2_flux_per_ch4_flux



CNTR       0.02
antro01    0.02
antro1     0.02
antro10    0.02
antro100   0.02
nemo       0.02
epia       0.02
munich     0.02
usdrydep   0.02
maud       0.02
zep        0.02
maxdep     0.02
Name: h2_flux_per_ch4_flux, dtype: float64

In [110]:
df_h2_flux_ssp_v2 = pd.DataFrame(data=[],columns=['SSP119','SSP434','SSP585'],index=['deltaH2','deltaCH4'])

for scen in df_h2_flux_ssp_v2.columns:
        df_h2_flux_ssp_v2[scen].loc['deltaH2'] = df_h2_flux_ssp['deltaH2'].loc['antro10_'+scen]
        df_h2_flux_ssp_v2[scen].loc['deltaCH4'] = df_h2_flux_ssp['deltaH2'].loc['10CH4_'+scen]
df_ch4_flux_ssp_v2

,SSP119,SSP434,SSP585
deltaH2,-2.90,-2.90,-3.13
deltaCH4,37.8,52.1,58.6


In [111]:
df_h2_flux_ssp_v2

,SSP119,SSP434,SSP585
deltaH2,9.79,9.77,9.77
deltaCH4,2.08,2.91,3.28


In [112]:
df_h2_flux_per_ch4_flux_ssp = -1.0*df_h2_flux_ssp_v2.loc['deltaCH4']/df_ch4_flux_ssp_v2.loc['deltaCH4']
df_h2_flux_per_ch4_flux_ssp.name = 'h2_flux_per_ch4_flux'
df_h2_flux_per_ch4_flux_ssp

SSP119   -0.06
SSP434   -0.06
SSP585   -0.06
Name: h2_flux_per_ch4_flux, dtype: object

### HYDROGEN AGWP100 CH4 [mW m-2 yr Tg-1]

agwp_ch4 = RF per flux H2

In [113]:
df_h2_agwp_ch4 = df_surf_ch4_per_h2_flux*spec_rf_ch4
df_h2_agwp_ch4.name = 'h2_agwp_ch4'
df_h2_agwp_ch4

CNTR        nan
antro01    0.43
antro1     0.43
antro10    0.43
antro100   0.41
nemo       0.50
epia       0.39
munich     0.41
usdrydep   0.36
maud       0.50
zep        0.41
maxdep     0.40
Name: h2_agwp_ch4, dtype: float64

In [114]:
#agwp_ch4 = RF per flux H2 (For the per flux table)
df_ch4_rf_per_h2_flux = df_surf_ch4_per_h2_flux*spec_rf_ch4
df_ch4_rf_per_h2_flux.name = 'ch4_rf_per_h2_flux'

df_ch4_rf_per_h2_flux

CNTR        nan
antro01    0.43
antro1     0.43
antro10    0.43
antro100   0.41
nemo       0.50
epia       0.39
munich     0.41
usdrydep   0.36
maud       0.50
zep        0.41
maxdep     0.40
Name: ch4_rf_per_h2_flux, dtype: float64

In [115]:
df_h2_agwp_ch4_ssp = df_surf_ch4_per_h2_flux_ssp*spec_rf_ch4
df_h2_agwp_ch4_ssp.name = 'h2_agwp_ch4'
df_h2_agwp_ch4_ssp

SSP119   0.43
SSP434   0.49
SSP585   0.51
Name: h2_agwp_ch4, dtype: object

In [116]:
#agwp_ch4 = RF per flux H2 (For the per flux table)
df_ch4_rf_per_h2_flux_ssp = df_surf_ch4_per_h2_flux_ssp*spec_rf_ch4
df_ch4_rf_per_h2_flux_ssp.name = 'ch4_rf_per_h2_flux'

df_ch4_rf_per_h2_flux_ssp

SSP119   0.43
SSP434   0.49
SSP585   0.51
Name: ch4_rf_per_h2_flux, dtype: object

### Initialize H2 GWP table

In [117]:
antmod = len(df_h2_agwp_ch4.index)
df_h2_gwp = pd.DataFrame(np.empty([5,antmod])*np.nan,columns=df_h2_agwp_ch4.index,
                         index=['O3','CH4','strat H2O','O3 CH4ind','strat H2O CH4ind'])
df_h2_gwp

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
O3,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
CH4,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
strat H2O,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
O3 CH4ind,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
strat H2O CH4ind,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [118]:
antmod = len(df_h2_agwp_ch4_ssp.index)
df_h2_gwp_ssp = pd.DataFrame(np.empty([5,antmod])*np.nan,columns=df_h2_agwp_ch4_ssp.index,
                         index=['O3','CH4','strat H2O','O3 CH4ind','strat H2O CH4ind'])
df_h2_gwp_ssp

,SSP119,SSP434,SSP585
O3,nan,nan,nan
CH4,nan,nan,nan
strat H2O,nan,nan,nan
O3 CH4ind,nan,nan,nan
strat H2O CH4ind,nan,nan,nan


### Add methane GWP

In [119]:
df_h2_gwp.loc['CH4'] = df_h2_agwp_ch4/agwp100_CO2
df_h2_gwp

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
O3,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
CH4,nan,4.78,4.82,4.80,4.56,5.56,4.33,4.63,3.97,5.57,4.57,4.42
strat H2O,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
O3 CH4ind,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
strat H2O CH4ind,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [120]:
df_h2_gwp_ssp.loc['CH4'] = df_h2_agwp_ch4_ssp/agwp100_CO2
df_h2_gwp_ssp

,SSP119,SSP434,SSP585
O3,nan,nan,nan
CH4,4.82,5.45,5.75
strat H2O,nan,nan,nan
O3 CH4ind,nan,nan,nan
strat H2O CH4ind,nan,nan,nan


### HYDROGEN AGWP100 strat H2O [mW m-2 yr Tg-1]

In [121]:
df_h2_agwp_h2o = df_h2o_rf.loc['10H2'].astype(float)/df_h2_flux['deltaH2']
df_h2_agwp_h2o

CNTR        nan
antro01    0.17
antro1     0.17
antro10    0.17
antro100   0.17
nemo       0.18
epia       0.14
munich     0.15
usdrydep   0.13
maud       0.18
zep        0.15
maxdep     0.16
dtype: float64

In [122]:
#Add to the flux table
df_h2o_rf_per_h2_flux = df_h2o_rf.loc['10H2'].astype(float)/df_h2_flux['deltaH2']
df_h2o_rf_per_h2_flux.name= 'h2o_rf_per_h2_flux'


#Strat H2O RF per methane flux 
df_h2o_rf_per_ch4_flux = df_h2o_rf.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_h2o_rf_per_ch4_flux.name = 'h2o_rf_per_ch4_flux'


In [123]:
df_h2_flux_ssp_v2

,SSP119,SSP434,SSP585
deltaH2,9.79,9.77,9.77
deltaCH4,2.08,2.91,3.28


In [124]:
df_h2_agwp_h2o_ssp = df_h2o_rf_ssp.loc['pertH2'].astype(float)/df_h2_flux_ssp_v2.loc['deltaH2']
df_h2_agwp_h2o_ssp = df_h2_agwp_h2o_ssp
#df_h2o_rf_ssp.loc['pertH2']
#df_h2_flux['deltaH2']
df_h2_agwp_h2o_ssp

SSP119   0.17
SSP434   0.18
SSP585   0.18
dtype: object

In [125]:
#Add to the flux table
df_h2o_rf_per_h2_flux_ssp = df_h2o_rf_ssp.loc['pertH2'].astype(float)/df_h2_flux_ssp_v2.loc['deltaH2']
df_h2o_rf_per_h2_flux_ssp.name= 'h2o_rf_per_h2_flux'


#Strat H2O RF per methane flux 
df_h2o_rf_per_ch4_flux_ssp = df_h2o_rf_ssp.loc['10CH4']/df_ch4_flux_ssp_v2.loc['deltaCH4']
df_h2o_rf_per_ch4_flux_ssp.name = 'h2o_rf_per_ch4_flux'


### Add stratospheric H2O GWP

In [126]:
df_h2_gwp.loc['strat H2O'] = df_h2_agwp_h2o/agwp100_CO2
df_h2_gwp

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
O3,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
CH4,nan,4.78,4.82,4.80,4.56,5.56,4.33,4.63,3.97,5.57,4.57,4.42
strat H2O,nan,1.87,1.87,1.88,1.90,2.05,1.57,1.69,1.45,2.05,1.65,1.80
O3 CH4ind,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
strat H2O CH4ind,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [127]:
df_h2_gwp_ssp.loc['strat H2O'] = df_h2_agwp_h2o_ssp/agwp100_CO2
df_h2_gwp_ssp

,SSP119,SSP434,SSP585
O3,nan,nan,nan
CH4,4.82,5.45,5.75
strat H2O,1.88,1.99,1.99
O3 CH4ind,nan,nan,nan
strat H2O CH4ind,nan,nan,nan


### HYDROGEN AGWP100 O3 [mW m-2 yr Tg-1]

In [128]:
df_h2_agwp_o3 = df_ozone_rf.loc['10H2'].astype(float)/df_h2_flux['deltaH2']
df_h2_agwp_o3.name = 'h2_agwp_o3'

df_h2_agwp_o3

CNTR        nan
antro01    0.19
antro1     0.21
antro10    0.22
antro100   0.22
nemo       0.23
epia       0.18
munich     0.20
usdrydep   0.17
maud       0.23
zep        0.19
maxdep     0.20
Name: h2_agwp_o3, dtype: float64

In [129]:
df_h2_agwp_o3_ssp = df_ozone_rf_ssp.loc['pertH2'].astype(float)/df_h2_flux_ssp_v2.loc['deltaH2']
df_h2_agwp_o3_ssp.name = 'h2_agwp_o3'

df_h2_agwp_o3_ssp

SSP119   0.22
SSP434   0.18
SSP585   0.18
Name: h2_agwp_o3, dtype: object

In [130]:
#Similar. To be used in the table:
df_ozone_rf_per_h2_flux = df_ozone_rf.loc['10H2'].astype(float)/df_h2_flux['deltaH2']
df_ozone_rf_per_h2_flux.name= 'ozone_rf_per_h2_flux'

df_ozone_rf_per_h2_flux


CNTR        nan
antro01    0.19
antro1     0.21
antro10    0.22
antro100   0.22
nemo       0.23
epia       0.18
munich     0.20
usdrydep   0.17
maud       0.23
zep        0.19
maxdep     0.20
Name: ozone_rf_per_h2_flux, dtype: float64

In [131]:
#Similar. To be used in the table:
df_ozone_rf_per_h2_flux_ssp = df_ozone_rf_ssp.loc['pertH2'].astype(float)/df_h2_flux_ssp_v2.loc['deltaH2']
df_ozone_rf_per_h2_flux_ssp.name= 'ozone_rf_per_h2_flux'

df_ozone_rf_per_h2_flux_ssp

SSP119   0.22
SSP434   0.18
SSP585   0.18
Name: ozone_rf_per_h2_flux, dtype: object

In [132]:
#Ozone RF per methane flux (move to the methane part?)
df_ozone_rf_per_ch4_flux = df_ozone_rf.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_ozone_rf_per_ch4_flux.name = 'ozone_rf_per_ch4_flux'
df_ozone_rf_per_ch4_flux

CNTR       0.82
antro01    0.82
antro1     0.82
antro10    0.82
antro100   0.82
nemo       0.82
epia       0.82
munich     0.82
usdrydep   0.82
maud       0.82
zep        0.82
maxdep     0.82
Name: ozone_rf_per_ch4_flux, dtype: object

In [133]:
#Ozone RF per methane flux (move to the methane part?)
df_ozone_rf_per_ch4_flux_ssp = df_ozone_rf_ssp.loc['10CH4']/df_ch4_flux_ssp_v2.loc['deltaCH4']
df_ozone_rf_per_ch4_flux_ssp.name = 'ozone_rf_per_ch4_flux'
df_ozone_rf_per_ch4_flux_ssp

SSP119   0.89
SSP434   0.84
SSP585   0.85
Name: ozone_rf_per_ch4_flux, dtype: object

### Add Ozone GWP

In [134]:
df_h2_gwp.loc['O3'] = df_h2_agwp_o3/agwp100_CO2
df_h2_gwp

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
O3,nan,2.13,2.38,2.42,2.41,2.59,2.06,2.22,1.89,2.62,2.14,2.27
CH4,nan,4.78,4.82,4.80,4.56,5.56,4.33,4.63,3.97,5.57,4.57,4.42
strat H2O,nan,1.87,1.87,1.88,1.90,2.05,1.57,1.69,1.45,2.05,1.65,1.80
O3 CH4ind,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
strat H2O CH4ind,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [135]:
df_h2_gwp_ssp.loc['O3'] = df_h2_agwp_o3_ssp/agwp100_CO2
df_h2_gwp_ssp

,SSP119,SSP434,SSP585
O3,2.49,2.06,2.02
CH4,4.82,5.45,5.75
strat H2O,1.88,1.99,1.99
O3 CH4ind,nan,nan,nan
strat H2O CH4ind,nan,nan,nan


### For the per flux table

In [136]:
df_ch4_flux


,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
YR13,725,725,724,722,696,724,724,724,724,724,724,724
deltaH2,0.00,-0.03,-0.30,-3.03,-28.7,-0.35,-0.27,-0.29,-0.25,-0.35,-0.29,-0.28
deltaCH4,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7,49.7


In [137]:
#df_trop_du_ozone_per_ch4_flux = df_ozone_du_trop.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
#df_trop_du_ozone_per_ch4_flux.name = 'trop_du_ozone_per_ch4_flux'

#df_strat_du_ozone_per_ch4_flux = df_ozone_du_strat.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
#df_strat_du_ozone_per_ch4_flux.name = 'strat_du_ozone_per_ch4_flux'

#df_trop_du_ozone_per_h2_flux = df_ozone_du_trop.loc['10H2']/df_h2_flux['deltaH2']
#df_trop_du_ozone_per_h2_flux.name = 'trop_du_ozone_per_h2_flux'

#df_strat_du_ozone_per_h2_flux = df_ozone_du_strat.loc['10H2']/df_h2_flux['deltaH2']
#df_strat_du_ozone_per_h2_flux.name = 'strat_du_ozone_per_h2_flux'

In [138]:
#df_trop_du_ozone_per_ch4_flux['OSLOCTM3-emi']=df_trop_du_ozone_per_ch4_flux['OSLOCTM3']
#df_strat_du_ozone_per_ch4_flux['OSLOCTM3-emi']=df_strat_du_ozone_per_ch4_flux['OSLOCTM3']

### HYDROGEN AGWP100 aerosol [mW m-2 yr Tg-1]

In [139]:
df_h2_agwp_aerosol = df_aerosol_rf.loc['10H2'].astype(float)/df_h2_flux['deltaH2']
df_h2_agwp_aerosol.name = 'h2_agwp_aerosol'
df_h2_agwp_aerosol
#NBNB GFDL-emi include methane induced.

CNTR       nan
antro01    nan
antro1     nan
antro10    nan
antro100   nan
nemo       nan
epia       nan
munich     nan
usdrydep   nan
maud       nan
zep        nan
maxdep     nan
Name: h2_agwp_aerosol, dtype: float64

In [140]:
df_h2_gwp.loc['aerosol'] = df_h2_agwp_aerosol/agwp100_CO2 

In [141]:
#Add to the flux table
df_aerosol_rf_per_h2_flux = df_aerosol_rf.loc['10H2'].astype(float)/df_h2_flux['deltaH2']
df_aerosol_rf_per_h2_flux.name= 'aerosol_rf_per_h2_flux'


df_aerosol_rf_per_ch4_flux = df_aerosol_rf.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_aerosol_rf_per_ch4_flux.name = 'aerosol_rf_per_ch4_flux'


In [142]:
df_aerosol_rf_per_ch4_flux

CNTR        NaN
antro01     NaN
antro1      NaN
antro10     NaN
antro100    NaN
nemo        NaN
epia        NaN
munich      NaN
usdrydep    NaN
maud        NaN
zep         NaN
maxdep      NaN
Name: aerosol_rf_per_ch4_flux, dtype: object

## Methane induced GWP:

### HYDROGEN AGWP100 methane induced O3 [mW m-2 yr Tg-1]

It does not matter here if we use surface concentration, burden or tropospheric concentration.

In [143]:
df_surf_ch4_per_h2_flux


CNTR        nan
antro01    1.11
antro1     1.12
antro10    1.12
antro100   1.06
nemo       1.29
epia       1.01
munich     1.08
usdrydep   0.92
maud       1.29
zep        1.06
maxdep     1.03
Name: surf_ch4_per_h2_flux, dtype: float64

In [144]:
df_surf_ch4_per_h2_flux_ssp

SSP119   1.12
SSP434   1.27
SSP585   1.34
Name: surf_ch4_per_h2_flux, dtype: object

In [145]:
#Wm-2/ppbCH4*ppbCH4/TgH2yr-1 -> Wm-2/TgH2yr-1
df_h2_agwp_ch4ind_o3 = df_ozone_rf.loc['10CH4'].astype(float)/df_ch4_surfconc.loc['delta']*df_surf_ch4_per_h2_flux
df_h2_agwp_ch4ind_o3.name = 'h2_agwp_ch4ind_o3'
df_h2_agwp_ch4ind_o3

CNTR        nan
antro01    0.25
antro1     0.25
antro10    0.25
antro100   0.24
nemo       0.29
epia       0.23
munich     0.24
usdrydep   0.21
maud       0.29
zep        0.24
maxdep     0.23
Name: h2_agwp_ch4ind_o3, dtype: float64

In [146]:
df_surf_ch4_per_h2_flux_ssp


SSP119   1.12
SSP434   1.27
SSP585   1.34
Name: surf_ch4_per_h2_flux, dtype: object

In [147]:
#Wm-2/ppbCH4*ppbCH4/TgH2yr-1 -> Wm-2/TgH2yr-1
df_h2_agwp_ch4ind_o3_ssp = df_ozone_rf_ssp.loc['10CH4'].astype(float)/df_ch4_surfconc_ssp_v2.loc['delta']*df_surf_ch4_per_h2_flux_ssp
df_h2_agwp_ch4ind_o3_ssp.name = 'h2_agwp_ch4ind_o3'
df_h2_agwp_ch4ind_o3_ssp

SSP119   0.26
SSP434   0.25
SSP585   0.27
Name: h2_agwp_ch4ind_o3, dtype: object

### Add methane induced O3 GWP

In [148]:
df_h2_gwp.loc['O3 CH4ind'] = df_h2_agwp_ch4ind_o3/agwp100_CO2
df_h2_gwp

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
O3,nan,2.13,2.38,2.42,2.41,2.59,2.06,2.22,1.89,2.62,2.14,2.27
CH4,nan,4.78,4.82,4.80,4.56,5.56,4.33,4.63,3.97,5.57,4.57,4.42
strat H2O,nan,1.87,1.87,1.88,1.90,2.05,1.57,1.69,1.45,2.05,1.65,1.80
O3 CH4ind,nan,2.79,2.81,2.80,2.66,3.25,2.53,2.70,2.32,3.25,2.67,2.58
strat H2O CH4ind,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
aerosol,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [149]:
df_h2_gwp_ssp.loc['O3 CH4ind'] = df_h2_agwp_ch4ind_o3_ssp/agwp100_CO2
df_h2_gwp_ssp

,SSP119,SSP434,SSP585
O3,2.49,2.06,2.02
CH4,4.82,5.45,5.75
strat H2O,1.88,1.99,1.99
O3 CH4ind,2.94,2.79,3.03
strat H2O CH4ind,nan,nan,nan


### HYDROGEN AGWP100 methane induced strat H2O [mW m-2 yr Tg-1]

In [150]:
df_h2_agwp_ch4ind_h2o = df_h2o_rf.loc['10CH4']/df_ch4_surfconc.loc['delta']*df_surf_ch4_per_h2_flux
df_h2_agwp_ch4ind_h2o.name = 'h2_agwp_ch4ind_h2o'
df_h2_agwp_ch4ind_h2o

CNTR        NaN
antro01    0.06
antro1     0.06
antro10    0.06
antro100   0.06
nemo       0.07
epia       0.05
munich     0.06
usdrydep   0.05
maud       0.07
zep        0.06
maxdep     0.05
Name: h2_agwp_ch4ind_h2o, dtype: object

In [151]:
df_h2_agwp_ch4ind_h2o_ssp = df_h2o_rf_ssp.loc['10CH4']/df_ch4_surfconc_ssp_v2.loc['delta']*df_surf_ch4_per_h2_flux_ssp
df_h2_agwp_ch4ind_h2o_ssp

SSP119   0.09
SSP434   0.10
SSP585   0.10
dtype: object

### Add methane induced strat H2O GWP

In [152]:
df_h2_gwp.loc['strat H2O CH4ind'] = df_h2_agwp_ch4ind_h2o/agwp100_CO2
df_h2_gwp

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
O3,nan,2.13,2.38,2.42,2.41,2.59,2.06,2.22,1.89,2.62,2.14,2.27
CH4,nan,4.78,4.82,4.80,4.56,5.56,4.33,4.63,3.97,5.57,4.57,4.42
strat H2O,nan,1.87,1.87,1.88,1.90,2.05,1.57,1.69,1.45,2.05,1.65,1.80
O3 CH4ind,nan,2.79,2.81,2.80,2.66,3.25,2.53,2.70,2.32,3.25,2.67,2.58
strat H2O CH4ind,nan,0.65,0.66,0.66,0.62,0.76,0.59,0.63,0.54,0.76,0.62,0.60
aerosol,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [153]:
df_h2_gwp_ssp.loc['strat H2O CH4ind'] = df_h2_agwp_ch4ind_h2o_ssp/agwp100_CO2
df_h2_gwp_ssp

,SSP119,SSP434,SSP585
O3,2.49,2.06,2.02
CH4,4.82,5.45,5.75
strat H2O,1.88,1.99,1.99
O3 CH4ind,2.94,2.79,3.03
strat H2O CH4ind,0.96,1.09,1.15


### HYDROGEN AGWP100 methane induced aerosols [mW m-2 yr Tg-1]

In [154]:
df_h2_agwp_ch4ind_aerosol = df_aerosol_rf.loc['10CH4']/df_ch4_surfconc.loc['delta']*df_surf_ch4_per_h2_flux
df_h2_agwp_ch4ind_aerosol.name = 'h2_agwp_ch4ind_aerosols'


### Hydrogen GWP including aerosols

In [155]:
df_h2_gwp.loc['aerosol CH4ind'] = df_h2_agwp_ch4ind_aerosol/agwp100_CO2
df_h2_gwp

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
O3,nan,2.13,2.38,2.42,2.41,2.59,2.06,2.22,1.89,2.62,2.14,2.27
CH4,nan,4.78,4.82,4.80,4.56,5.56,4.33,4.63,3.97,5.57,4.57,4.42
strat H2O,nan,1.87,1.87,1.88,1.90,2.05,1.57,1.69,1.45,2.05,1.65,1.80
O3 CH4ind,nan,2.79,2.81,2.80,2.66,3.25,2.53,2.70,2.32,3.25,2.67,2.58
strat H2O CH4ind,nan,0.65,0.66,0.66,0.62,0.76,0.59,0.63,0.54,0.76,0.62,0.60
aerosol,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
aerosol CH4ind,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [156]:
#Not include the aerosol GWP in the main table. Drop them here
df_h2_gwp = df_h2_gwp.drop(['aerosol','aerosol CH4ind'])
df_h2_gwp

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
O3,nan,2.13,2.38,2.42,2.41,2.59,2.06,2.22,1.89,2.62,2.14,2.27
CH4,nan,4.78,4.82,4.80,4.56,5.56,4.33,4.63,3.97,5.57,4.57,4.42
strat H2O,nan,1.87,1.87,1.88,1.90,2.05,1.57,1.69,1.45,2.05,1.65,1.80
O3 CH4ind,nan,2.79,2.81,2.80,2.66,3.25,2.53,2.70,2.32,3.25,2.67,2.58
strat H2O CH4ind,nan,0.65,0.66,0.66,0.62,0.76,0.59,0.63,0.54,0.76,0.62,0.60


In [157]:
df_h2_gwp_ssp

,SSP119,SSP434,SSP585
O3,2.49,2.06,2.02
CH4,4.82,5.45,5.75
strat H2O,1.88,1.99,1.99
O3 CH4ind,2.94,2.79,3.03
strat H2O CH4ind,0.96,1.09,1.15


# Methane GWP

Initialize CH4 GWP

In [158]:
antmod = len(df_h2_agwp_ch4.index)
df_ch4_gwp = pd.DataFrame(np.empty([4,antmod])*np.nan,columns=df_h2_agwp_ch4.index,
                         index=['O3','CH4','strat H2O','H2'])
df_ch4_gwp

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
O3,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
CH4,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
strat H2O,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
H2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [159]:
antmod = len(df_h2_agwp_ch4_ssp.index)
df_ch4_gwp_ssp = pd.DataFrame(np.empty([4,antmod])*np.nan,columns=df_h2_agwp_ch4_ssp.index,
                         index=['O3','CH4','strat H2O','H2'])
df_ch4_gwp_ssp

,SSP119,SSP434,SSP585
O3,nan,nan,nan
CH4,nan,nan,nan
strat H2O,nan,nan,nan
H2,nan,nan,nan


### Methane AGWP100 O3 [mW m-2 yr Tg-1]

In [160]:
df_ch4_agwp_o3 = df_ozone_rf.loc['10CH4']/df_ch4_surfconc.loc['delta']*df_surf_ch4_per_ch4_flux
df_ch4_agwp_o3.name = 'ch4_agwp_o3'
df_ch4_agwp_o3

CNTR       0.82
antro01    0.82
antro1     0.82
antro10    0.82
antro100   0.82
nemo       0.82
epia       0.82
munich     0.82
usdrydep   0.82
maud       0.82
zep        0.82
maxdep     0.82
Name: ch4_agwp_o3, dtype: object

In [161]:
df_ch4_agwp_o3_ssp = df_ozone_rf_ssp.loc['10CH4']/df_ch4_surfconc_ssp_v2.loc['delta']*df_surf_ch4_per_ch4_flux_ssp
df_ch4_agwp_o3_ssp.name = 'ch4_agwp_o3'
df_ch4_agwp_o3_ssp

SSP119   0.89
SSP434   0.84
SSP585   0.85
Name: ch4_agwp_o3, dtype: object

In [162]:
test = df_ozone_rf.loc['10CH4']/df_ch4_surfconc.loc['delta']*df_surf_ch4_per_ch4_flux
test

CNTR       0.82
antro01    0.82
antro1     0.82
antro10    0.82
antro100   0.82
nemo       0.82
epia       0.82
munich     0.82
usdrydep   0.82
maud       0.82
zep        0.82
maxdep     0.82
dtype: object

In [163]:
test2 = df_ozone_rf.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
#test2 is equal test
test2

CNTR       0.82
antro01    0.82
antro1     0.82
antro10    0.82
antro100   0.82
nemo       0.82
epia       0.82
munich     0.82
usdrydep   0.82
maud       0.82
zep        0.82
maxdep     0.82
dtype: object

In [164]:
test2 = df_ozone_rf_ssp.loc['10CH4']/df_ch4_flux_ssp_v2.loc['deltaCH4']
#test2 is equal test
test2

SSP119   0.89
SSP434   0.84
SSP585   0.85
dtype: object

### Add ozone GWP

In [165]:
df_ch4_gwp.loc['O3'] =df_ch4_agwp_o3/agwp100_CO2 

In [166]:
df_ch4_gwp_ssp.loc['O3'] =df_ch4_agwp_o3_ssp/agwp100_CO2 

### Methane AGWP100 Methane [mW m-2 yr Tg-1]

In [167]:
df_ch4_agwp =df_surf_ch4_per_ch4_flux*spec_rf_ch4
df_ch4_agwp.name = 'ch4_agwp'


In [168]:
df_ch4_agwp_ssp =df_surf_ch4_per_ch4_flux_ssp*spec_rf_ch4
df_ch4_agwp_ssp.name = 'ch4_agwp'

### Add methane GWP

In [169]:
#Add Methane GWP:
df_ch4_gwp.loc['CH4'] =df_ch4_agwp/agwp100_CO2 

In [170]:
df_ch4_gwp_ssp.loc['CH4'] =df_ch4_agwp_ssp/agwp100_CO2 

### Methane AGWP100 strat H2O [mW m-2 yr Tg-1]

In [171]:
df_ch4_agwp_h2o = df_h2o_rf.loc['10CH4']/df_ch4_surfconc.loc['delta']*df_surf_ch4_per_ch4_flux
df_ch4_agwp_h2o.name = 'ch4_agwp_h2o'

In [172]:
df_ch4_agwp_h2o_ssp = df_h2o_rf_ssp.loc['10CH4']/df_ch4_surfconc_ssp_v2.loc['delta']*df_surf_ch4_per_ch4_flux_ssp
df_ch4_agwp_h2o_ssp 

SSP119   0.29
SSP434   0.33
SSP585   0.32
dtype: object

### Add Strat H2O GWP:

In [173]:
df_ch4_gwp.loc['strat H2O'] = df_ch4_agwp_h2o/agwp100_CO2

In [174]:
df_ch4_gwp_ssp.loc['strat H2O'] = df_ch4_agwp_h2o_ssp/agwp100_CO2

### Methane AGWP100 aerosols [mW m-2 yr Tg-1]

In [175]:
df_ch4_agwp_aerosol = df_aerosol_rf.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_ch4_agwp_aerosol.name = 'ch4_agwp_aerosol'


### Add Aerosol GWP

In [176]:
df_ch4_gwp.loc['aerosol'] = df_ch4_agwp_aerosol/agwp100_CO2
df_ch4_gwp

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
O3,9.17,9.17,9.17,9.17,9.17,9.17,9.17,9.17,9.17,9.17,9.17,9.17
CH4,15.7,15.7,15.7,15.7,15.7,15.7,15.7,15.7,15.7,15.7,15.7,15.7
strat H2O,2.14,2.14,2.14,2.14,2.14,2.14,2.14,2.14,2.14,2.14,2.14,2.14
H2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
aerosol,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [177]:
#And drop the aerosol GWP in the main table:
df_ch4_gwp = df_ch4_gwp.drop(['aerosol'])
df_ch4_gwp

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
O3,9.17,9.17,9.17,9.17,9.17,9.17,9.17,9.17,9.17,9.17,9.17,9.17
CH4,15.7,15.7,15.7,15.7,15.7,15.7,15.7,15.7,15.7,15.7,15.7,15.7
strat H2O,2.14,2.14,2.14,2.14,2.14,2.14,2.14,2.14,2.14,2.14,2.14,2.14
H2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [178]:
df_ch4_gwp_ssp

,SSP119,SSP434,SSP585
O3,9.90,9.41,9.47
CH4,16.3,18.4,18.0
strat H2O,3.22,3.67,3.60
H2,nan,nan,nan


### Add GWP via H2

In [179]:
#df_ch4_gwp.loc['H2'] = df_h2_flux_per_ch4_flux*df_h2_gwp.sum()
#df_ch4_gwp_ssp.loc['H2'] = df_h2_flux_per_ch4_flux_ssp*df_h2_gwp_ssp.sum()


# Part III: Main results and tables

## H2 GWP 100

In [180]:
#model_dict = { 'OSLOCTM3':'OsloCTM',
#               'WACCM6-2deg':'WACCM',
#               'INCA':'INCA',
#               'GFDL-emi':'GFDL-emi',
#               'GFDL_nudge':'GFDL',
#               'UKCA':'UKCA',
#               'OSLOCTM3-emi':'OsloCTM-emi'}
#
#sorted_array = ['GFDL','INCA','OsloCTM','UKCA','WACCM','GFDL-emi','OsloCTM-emi']
#sorted_array_2 = ['GFDL','INCA','OsloCTM','UKCA','WACCM','GFDL-emi','OsloCTM-emi','Model mean']

In [181]:
df_h2_gwp = pd.concat([df_h2_gwp,df_h2_gwp_ssp],axis=1, join='inner',sort=False)
df_h2_gwp
                      

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep,SSP119,SSP434,SSP585
O3,nan,2.13,2.38,2.42,2.41,2.59,2.06,2.22,1.89,2.62,2.14,2.27,2.49,2.06,2.02
CH4,nan,4.78,4.82,4.80,4.56,5.56,4.33,4.63,3.97,5.57,4.57,4.42,4.82,5.45,5.75
strat H2O,nan,1.87,1.87,1.88,1.90,2.05,1.57,1.69,1.45,2.05,1.65,1.80,1.88,1.99,1.99
O3 CH4ind,nan,2.79,2.81,2.80,2.66,3.25,2.53,2.70,2.32,3.25,2.67,2.58,2.94,2.79,3.03
strat H2O CH4ind,nan,0.65,0.66,0.66,0.62,0.76,0.59,0.63,0.54,0.76,0.62,0.60,0.96,1.09,1.15


In [182]:
#df_h2_gwp = df_h2_gwp[sorted(df_h2_gwp.columns)]
df_h2_gwp = df_h2_gwp.drop('CNTR',axis=1)
df_h2_gwp.loc['total']=df_h2_gwp.sum()

df_h2_gwp_table = df_h2_gwp.copy()

#df_h2_gwp_table.rename(model_dict,axis=1,inplace=True)
#df_h2_gwp_table = df_h2_gwp_table[sorted_array]
df_h2_gwp_table.T


,O3,CH4,strat H2O,O3 CH4ind,strat H2O CH4ind,total
antro01,2.13,4.78,1.87,2.79,0.65,12.2
antro1,2.38,4.82,1.87,2.81,0.66,12.5
antro10,2.42,4.80,1.88,2.80,0.66,12.6
antro100,2.41,4.56,1.90,2.66,0.62,12.2
nemo,2.59,5.56,2.05,3.25,0.76,14.2
epia,2.06,4.33,1.57,2.53,0.59,11.1
munich,2.22,4.63,1.69,2.70,0.63,11.9
usdrydep,1.89,3.97,1.45,2.32,0.54,10.2
maud,2.62,5.57,2.05,3.25,0.76,14.2
zep,2.14,4.57,1.65,2.67,0.62,11.7


## CH4 GWP 100

In [183]:
df_ch4_gwp = pd.concat([df_ch4_gwp,df_ch4_gwp_ssp],axis=1, join='inner',sort=False)
df_ch4_gwp

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep,SSP119,SSP434,SSP585
O3,9.17,9.17,9.17,9.17,9.17,9.17,9.17,9.17,9.17,9.17,9.17,9.17,9.90,9.41,9.47
CH4,15.7,15.7,15.7,15.7,15.7,15.7,15.7,15.7,15.7,15.7,15.7,15.7,16.3,18.4,18.0
strat H2O,2.14,2.14,2.14,2.14,2.14,2.14,2.14,2.14,2.14,2.14,2.14,2.14,3.22,3.67,3.60
H2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [184]:
#df_ch4_gwp = df_ch4_gwp[sorted(df_ch4_gwp.columns)]
#df_ch4_gwp.drop('GFDL-emi',axis=1).to_csv(outputpath + 'table_ch4_gwp.csv')

df_ch4_gwp = df_ch4_gwp.drop('CNTR',axis=1)
df_ch4_gwp.to_csv(outputpath + 'table_ch4_gwp.csv')
df_ch4_gwp.loc['total']=df_ch4_gwp.sum()
df_ch4_gwp.T

,O3,CH4,strat H2O,H2,total
antro01,9.17,15.7,2.14,nan,27.0
antro1,9.17,15.7,2.14,nan,27.0
antro10,9.17,15.7,2.14,nan,27.0
antro100,9.17,15.7,2.14,nan,27.0
nemo,9.17,15.7,2.14,nan,27.0
epia,9.17,15.7,2.14,nan,27.0
munich,9.17,15.7,2.14,nan,27.0
usdrydep,9.17,15.7,2.14,nan,27.0
maud,9.17,15.7,2.14,nan,27.0
zep,9.17,15.7,2.14,nan,27.0


## Table per flux H2

In [185]:
df_per_flux_h2 = pd.concat([df_h2_flux['deltaH2'],
                            df_surf_h2_per_h2_flux,
                            df_surf_ch4_per_h2_flux,
                            df_ch4_flux_per_h2_flux,
                            df_ch4_rf_per_h2_flux,
                            #df_trop_du_ozone_per_h2_flux*1000.,
                            #df_strat_du_ozone_per_h2_flux*1000.,
                            df_ozone_rf_per_h2_flux,
                            df_h2o_rf_per_h2_flux,
                            df_aerosol_rf_per_h2_flux],axis=1, sort=False)

df_per_flux_h2

,deltaH2,surf_h2_per_h2_flux,surf_ch4_per_h2_flux,ch4_flux_per_h2_flux,ch4_rf_per_h2_flux,ozone_rf_per_h2_flux,h2o_rf_per_h2_flux,aerosol_rf_per_h2_flux
CNTR,0.00,nan,nan,nan,nan,nan,nan,nan
antro01,0.10,6.27,1.11,0.30,0.43,0.19,0.17,nan
antro1,0.99,6.27,1.12,0.31,0.43,0.21,0.17,nan
antro10,9.90,6.28,1.12,0.31,0.43,0.22,0.17,nan
antro100,99.1,6.35,1.06,0.29,0.41,0.22,0.17,nan
nemo,0.99,7.92,1.29,0.35,0.50,0.23,0.18,nan
epia,0.99,6.05,1.01,0.28,0.39,0.18,0.14,nan
munich,0.99,6.32,1.08,0.29,0.41,0.20,0.15,nan
usdrydep,0.99,5.20,0.92,0.25,0.36,0.17,0.13,nan
maud,0.99,9.24,1.29,0.35,0.50,0.23,0.18,nan


In [186]:
df_per_flux_h2.loc['SSP119'] = None
df_per_flux_h2.loc['SSP434'] = None
df_per_flux_h2.loc['SSP585'] = None

scen_list = ['SSP119','SSP434','SSP585']
df_h2_flux_ssp_v2
for scen in scen_list:
    df_per_flux_h2['deltaH2'].loc[scen] = df_h2_flux_ssp_v2.loc['deltaH2'][scen]
    df_per_flux_h2['surf_h2_per_h2_flux'].loc[scen] = df_surf_h2_per_h2_flux_ssp['antro10_'+scen]
    df_per_flux_h2['surf_ch4_per_h2_flux'].loc[scen] = df_surf_ch4_per_h2_flux_ssp[scen]
    df_per_flux_h2['ch4_flux_per_h2_flux'].loc[scen] = df_ch4_flux_per_h2_flux_ssp[scen]
    df_per_flux_h2['ch4_rf_per_h2_flux'].loc[scen] = df_ch4_rf_per_h2_flux_ssp[scen]
    df_per_flux_h2['ozone_rf_per_h2_flux'].loc[scen] = df_ozone_rf_per_h2_flux_ssp[scen]
    df_per_flux_h2['h2o_rf_per_h2_flux'].loc[scen] = df_h2o_rf_per_h2_flux_ssp[scen]
df_per_flux_h2

,deltaH2,surf_h2_per_h2_flux,surf_ch4_per_h2_flux,ch4_flux_per_h2_flux,ch4_rf_per_h2_flux,ozone_rf_per_h2_flux,h2o_rf_per_h2_flux,aerosol_rf_per_h2_flux
CNTR,0.00,nan,nan,nan,nan,nan,nan,nan
antro01,0.10,6.27,1.11,0.30,0.43,0.19,0.17,nan
antro1,0.99,6.27,1.12,0.31,0.43,0.21,0.17,nan
antro10,9.90,6.28,1.12,0.31,0.43,0.22,0.17,nan
antro100,99.1,6.35,1.06,0.29,0.41,0.22,0.17,nan
nemo,0.99,7.92,1.29,0.35,0.50,0.23,0.18,nan
epia,0.99,6.05,1.01,0.28,0.39,0.18,0.14,nan
munich,0.99,6.32,1.08,0.29,0.41,0.20,0.15,nan
usdrydep,0.99,5.20,0.92,0.25,0.36,0.17,0.13,nan
maud,0.99,9.24,1.29,0.35,0.50,0.23,0.18,nan


In [187]:
#Save to file:
df_per_flux_h2 = df_per_flux_h2.sort_index()
df_per_flux_h2.to_csv(outputpath + 'table_per_flux_h2.csv')

#Rename the columns:
columns_names={'deltaH2':'Flux H2 [Tg/yr]',
               'surf_h2_per_h2_flux': 'Surf. conc. H2 per flux [ppb yr/Tg]',
               'surf_ch4_per_h2_flux':'Surf. conc. CH4 per flux [ppb yr/Tg]',
               'ch4_flux_per_h2_flux':'Flux CH4/Flux H2 [Tg CH4/Tg H2]',
               'ch4_rf_per_h2_flux':'CH4 RF per flux [mW m-2 yr/ Tg]',
               'trop_du_ozone_per_h2_flux':'Trop. ozone per flux [10$^{-3}$ DU yr/Tg]',
               'strat_du_ozone_per_h2_flux':'Strat. ozone per flux [10$^{-3}$ DU yr/Tg]',
               'ozone_rf_per_h2_flux':'ozone RF per flux [mW m-2 yr/ Tg]',
               'h2o_rf_per_h2_flux':'Strat. H2O RF per flux [mW m-2 yr/ Tg]',
               'aerosol_rf_per_h2_flux':'Aerosol RF per flux [mW m-2 yr/ Tg]'}

#Rename column names:
df_per_flux_h2.rename(columns=dict(columns_names),inplace=True) #[df_per_flux_h2.columns])
df_per_flux_h2

,Flux H2 [Tg/yr],Surf. conc. H2 per flux [ppb yr/Tg],Surf. conc. CH4 per flux [ppb yr/Tg],Flux CH4/Flux H2 [Tg CH4/Tg H2],CH4 RF per flux [mW m-2 yr/ Tg],ozone RF per flux [mW m-2 yr/ Tg],Strat. H2O RF per flux [mW m-2 yr/ Tg],Aerosol RF per flux [mW m-2 yr/ Tg]
CNTR,0.00,nan,nan,nan,nan,nan,nan,nan
SSP119,9.79,6.36,1.12,0.30,0.43,0.22,0.17,nan
SSP434,9.77,6.52,1.27,0.30,0.49,0.18,0.18,nan
SSP585,9.77,6.42,1.34,0.32,0.51,0.18,0.18,nan
antro01,0.10,6.27,1.11,0.30,0.43,0.19,0.17,nan
antro1,0.99,6.27,1.12,0.31,0.43,0.21,0.17,nan
antro10,9.90,6.28,1.12,0.31,0.43,0.22,0.17,nan
antro100,99.1,6.35,1.06,0.29,0.41,0.22,0.17,nan
epia,0.99,6.05,1.01,0.28,0.39,0.18,0.14,nan
maud,0.99,9.24,1.29,0.35,0.50,0.23,0.18,nan


## Table per flux CH4

In [188]:
df_per_flux_ch4 = pd.concat([df_ch4_flux.loc['deltaCH4'],
                            df_surf_ch4_per_ch4_flux,
                            df_h2_flux_per_ch4_flux,
                            #df_trop_du_ozone_per_ch4_flux*1000.,
                            #df_strat_du_ozone_per_ch4_flux*1000.,
                            df_ozone_rf_per_ch4_flux,
                            df_h2o_rf_per_ch4_flux,
                            df_aerosol_rf_per_ch4_flux],axis=1,sort=False)


df_per_flux_ch4        

,deltaCH4,surf_ch4_per_ch4_flux,h2_flux_per_ch4_flux,ozone_rf_per_ch4_flux,h2o_rf_per_ch4_flux,aerosol_rf_per_ch4_flux
CNTR,49.7,3.65,0.02,0.82,0.19,NaN
antro01,49.7,3.65,0.02,0.82,0.19,NaN
antro1,49.7,3.65,0.02,0.82,0.19,NaN
antro10,49.7,3.65,0.02,0.82,0.19,NaN
antro100,49.7,3.65,0.02,0.82,0.19,NaN
nemo,49.7,3.65,0.02,0.82,0.19,NaN
epia,49.7,3.65,0.02,0.82,0.19,NaN
munich,49.7,3.65,0.02,0.82,0.19,NaN
usdrydep,49.7,3.65,0.02,0.82,0.19,NaN
maud,49.7,3.65,0.02,0.82,0.19,NaN


In [189]:
df_per_flux_ch4.loc['SSP119'] = None
df_per_flux_ch4.loc['SSP434'] = None
df_per_flux_ch4.loc['SSP585'] = None

In [190]:
df_h2o_rf_per_ch4_flux_ssp

SSP119   0.29
SSP434   0.33
SSP585   0.32
Name: h2o_rf_per_ch4_flux, dtype: object

In [191]:
scen_list = ['SSP119','SSP434','SSP585']

for scen in scen_list:
    df_per_flux_ch4['deltaCH4'].loc[scen] = df_ch4_flux_ssp_v2.loc['deltaCH4'][scen]
    df_per_flux_ch4['surf_ch4_per_ch4_flux'].loc[scen] = df_surf_ch4_per_ch4_flux_ssp[scen]
    df_per_flux_ch4['h2_flux_per_ch4_flux'].loc[scen] =df_h2_flux_per_ch4_flux_ssp[scen]
    df_per_flux_ch4['ozone_rf_per_ch4_flux'].loc[scen] = df_ozone_rf_per_ch4_flux_ssp[scen]
    df_per_flux_ch4['h2o_rf_per_ch4_flux'].loc[scen] = df_h2o_rf_per_ch4_flux_ssp[scen]
df_per_flux_ch4

/home/ragnhibs/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,deltaCH4,surf_ch4_per_ch4_flux,h2_flux_per_ch4_flux,ozone_rf_per_ch4_flux,h2o_rf_per_ch4_flux,aerosol_rf_per_ch4_flux
CNTR,49.7,3.65,0.02,0.82,0.19,NaN
antro01,49.7,3.65,0.02,0.82,0.19,NaN
antro1,49.7,3.65,0.02,0.82,0.19,NaN
antro10,49.7,3.65,0.02,0.82,0.19,NaN
antro100,49.7,3.65,0.02,0.82,0.19,NaN
nemo,49.7,3.65,0.02,0.82,0.19,NaN
epia,49.7,3.65,0.02,0.82,0.19,NaN
munich,49.7,3.65,0.02,0.82,0.19,NaN
usdrydep,49.7,3.65,0.02,0.82,0.19,NaN
maud,49.7,3.65,0.02,0.82,0.19,NaN


In [192]:
#Save to file:
#df_per_flux_ch4 = df_per_flux_ch4.sort_index()
df_per_flux_ch4.to_csv(outputpath + 'table_per_flux_ch4.csv')

#Rename the columns:
columns_names={'deltaCH4':'Flux CH4 [Tg/yr]',
               'surf_ch4_per_ch4_flux':'Surf. conc. CH4 per flux [ppb yr/Tg]',
               'h2_flux_per_ch4_flux':'Flux H2/Flux CH4 [Tg H2/Tg CH4]',
               'trop_du_ozone_per_ch4_flux':'Trop. ozone per flux [10$^{-3}$ DU yr/Tg]',
               'strat_du_ozone_per_ch4_flux':'Strat. ozone per flux [10$^{-3}$ DU yr/Tg]',
               'ozone_rf_per_ch4_flux':'ozone RF per flux [mW m-2 yr/ Tg]',
               'h2o_rf_per_ch4_flux':'Strat H2O RF per flux [mW m-2 yr/ Tg]',
               'aerosol_rf_per_ch4_flux':'Aerosol RF per flux [mW m-2 yr/ Tg]'}
               
#Rename column names:

df_per_flux_ch4.rename(columns=dict(columns_names),inplace=True) 

## Table per flux H2 (including changes in methane)

In [193]:
#Reread - to get the other heading.

df_per_flux_h2_combined = pd.read_csv(outputpath + 'table_per_flux_h2.csv',index_col=0)
df_per_flux_h2_combined

,deltaH2,surf_h2_per_h2_flux,surf_ch4_per_h2_flux,ch4_flux_per_h2_flux,ch4_rf_per_h2_flux,ozone_rf_per_h2_flux,h2o_rf_per_h2_flux,aerosol_rf_per_h2_flux
CNTR,0.00,nan,nan,nan,nan,nan,nan,nan
SSP119,9.79,6.36,1.12,0.30,0.43,0.22,0.17,nan
SSP434,9.77,6.52,1.27,0.30,0.49,0.18,0.18,nan
SSP585,9.77,6.42,1.34,0.32,0.51,0.18,0.18,nan
antro01,0.10,6.27,1.11,0.30,0.43,0.19,0.17,nan
antro1,0.99,6.27,1.12,0.31,0.43,0.21,0.17,nan
antro10,9.90,6.28,1.12,0.31,0.43,0.22,0.17,nan
antro100,99.1,6.35,1.06,0.29,0.41,0.22,0.17,nan
epia,0.99,6.05,1.01,0.28,0.39,0.18,0.14,nan
maud,0.99,9.24,1.29,0.35,0.50,0.23,0.18,nan


In [194]:
df_per_flux_ch4_add  = pd.read_csv(outputpath + 'table_per_flux_ch4.csv',index_col=0)
df_per_flux_ch4_add

,deltaCH4,surf_ch4_per_ch4_flux,h2_flux_per_ch4_flux,ozone_rf_per_ch4_flux,h2o_rf_per_ch4_flux,aerosol_rf_per_ch4_flux
CNTR,49.7,3.65,0.02,0.82,0.19,nan
antro01,49.7,3.65,0.02,0.82,0.19,nan
antro1,49.7,3.65,0.02,0.82,0.19,nan
antro10,49.7,3.65,0.02,0.82,0.19,nan
antro100,49.7,3.65,0.02,0.82,0.19,nan
nemo,49.7,3.65,0.02,0.82,0.19,nan
epia,49.7,3.65,0.02,0.82,0.19,nan
munich,49.7,3.65,0.02,0.82,0.19,nan
usdrydep,49.7,3.65,0.02,0.82,0.19,nan
maud,49.7,3.65,0.02,0.82,0.19,nan


In [195]:
frac = df_per_flux_h2_combined['ch4_flux_per_h2_flux'] #Tg CH4/Tg H2 /Tg CH4 = 1/Tg H2
frac

CNTR        nan
SSP119     0.30
SSP434     0.30
SSP585     0.32
antro01    0.30
antro1     0.31
antro10    0.31
antro100   0.29
epia       0.28
maud       0.35
maxdep     0.28
munich     0.29
nemo       0.35
usdrydep   0.25
zep        0.29
Name: ch4_flux_per_h2_flux, dtype: float64

In [196]:
df_per_flux_ch4_add
#Keep the following:
#deltaH2
#surf_h2_per_h2_flux keep as h2_flux_per_ch4_flux small
#surf_ch4_per_h2_flux
#ch4_flux_per_h2_flux
#ch4_rf_per_h2_flux

#add:
#trop_du_ozone_per_h2_flux
#strat_du_ozone_per_h2_flux
#ozone_rf_per_h2_flux
#h2o_rf_per_h2_flux
#aerosol_rf_per_h2_flux

#df_per_flux_h2_combined['trop_du_ozone_per_h2_flux'] = df_per_flux_h2_combined['trop_du_ozone_per_h2_flux'] + df_per_flux_ch4_add['trop_du_ozone_per_ch4_flux']*frac
#df_per_flux_h2_combined['strat_du_ozone_per_h2_flux'] = df_per_flux_h2_combined['strat_du_ozone_per_h2_flux'] + df_per_flux_ch4_add['strat_du_ozone_per_ch4_flux']*frac
df_per_flux_h2_combined['ozone_rf_per_h2_flux'] = df_per_flux_h2_combined['ozone_rf_per_h2_flux'] + df_per_flux_ch4_add['ozone_rf_per_ch4_flux']*frac
df_per_flux_h2_combined['h2o_rf_per_h2_flux'] = df_per_flux_h2_combined['h2o_rf_per_h2_flux'] + df_per_flux_ch4_add['h2o_rf_per_ch4_flux']*frac
df_per_flux_h2_combined['aerosol_rf_per_h2_flux'] = df_per_flux_h2_combined['aerosol_rf_per_h2_flux'] + df_per_flux_ch4_add['aerosol_rf_per_ch4_flux']*frac


#df_per_flux_h2_combined

#Save to file:
#df_per_flux_h2_combined = df_per_flux_h2_combined.sort_index()
df_per_flux_h2_combined.to_csv(outputpath + 'table_per_flux_h2_combined.csv')


In [197]:
#Rename the columns:
columns_names={'deltaH2':'Flux H2 [Tg/yr]',
               'surf_h2_per_h2_flux': 'Surf. conc. H2 per flux [ppb yr/Tg]',
               'surf_ch4_per_h2_flux':'Surf. conc. CH4 per flux [ppb yr/Tg]',
               'ch4_flux_per_h2_flux':'Flux CH4/Flux H2 [Tg CH4/Tg H2]',
               'ch4_rf_per_h2_flux':'CH4 RF per flux [mW m-2 yr/ Tg]',
               'trop_du_ozone_per_h2_flux':'Trop. ozone per flux [10$^{-3}$ DU yr/Tg]',
               'strat_du_ozone_per_h2_flux':'Strat. ozone per flux [10$^{-3}$ DU yr/Tg]',
               'ozone_rf_per_h2_flux':'ozone RF per flux [mW m-2 yr/ Tg]',
               'h2o_rf_per_h2_flux':'Strat. H2O RF per flux [mW m-2 yr/ Tg]',
               'aerosol_rf_per_h2_flux':'Aerosol RF per flux [mW m-2 yr/ Tg]'}
#Rename column names:
df_per_flux_h2_combined.rename(columns=dict(columns_names),inplace=True) #[df_per_flux_h2.columns])
#df_per_flux_h2_combined.rename(model_dict,axis=0,inplace=True)
#df_per_flux_h2_combined.loc['Model mean'] = df_per_flux_h2_combined.drop(['GFDL-emi','OsloCTM-emi']).mean()
#df_per_flux_h2_combined['Flux H2 [Tg/yr]'].loc['Model mean']=np.nan
#df_per_flux_h2_combined=df_per_flux_h2_combined.reindex(sorted_array_2)
df_per_flux_h2_combined

,Flux H2 [Tg/yr],Surf. conc. H2 per flux [ppb yr/Tg],Surf. conc. CH4 per flux [ppb yr/Tg],Flux CH4/Flux H2 [Tg CH4/Tg H2],CH4 RF per flux [mW m-2 yr/ Tg],ozone RF per flux [mW m-2 yr/ Tg],Strat. H2O RF per flux [mW m-2 yr/ Tg],Aerosol RF per flux [mW m-2 yr/ Tg]
CNTR,0.00,nan,nan,nan,nan,nan,nan,nan
SSP119,9.79,6.36,1.12,0.30,0.43,0.49,0.25,nan
SSP434,9.77,6.52,1.27,0.30,0.49,0.43,0.28,nan
SSP585,9.77,6.42,1.34,0.32,0.51,0.45,0.28,nan
antro01,0.10,6.27,1.11,0.30,0.43,0.44,0.23,nan
antro1,0.99,6.27,1.12,0.31,0.43,0.47,0.23,nan
antro10,9.90,6.28,1.12,0.31,0.43,0.47,0.23,nan
antro100,99.1,6.35,1.06,0.29,0.41,0.45,0.23,nan
epia,0.99,6.05,1.01,0.28,0.39,0.41,0.19,nan
maud,0.99,9.24,1.29,0.35,0.50,0.53,0.25,nan


## H2 budget table

In [198]:
df_h2_burden


,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
YR13,205,205,207,226,423,207,207,207,206,207,207,207
deltaH2,0.00,0.22,2.15,21.5,218,2.53,1.93,2.05,1.76,2.56,2.07,1.97


In [255]:
df_h2_surfconc

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
YR13,559,560,565,621,"1,189",567,565,565,564,568,566,565
delta,0.00,0.62,6.22,62.2,630,7.85,6.01,6.23,5.15,9.15,6.63,5.42


In [256]:
df_budget_h2 = pd.concat([df_h2_burden.iloc[0],
                          df_h2_surfconc.iloc[0],
                          df_h2_atmloss,
                          df_h2_atmprod,
                          df_h2_drydep,
                          df_h2_estemis, 
                          df_h2_atm_lifetime,
                          df_h2_soil_sink_lifetime,
                          df_h2_lifetime
                         ],axis=1)

df_budget_h2.columns = ['H2 burden [Tg]',
                        'H2 surf. conc [ppbv]',
                        'H2 atm loss [Tg/yr]',
                        'H2 atm prod [Tg/yr]',
                        'H2 soil sink [Tg/yr]',
                        'H2 estimated emissions [Tg/yr]',
                        'H2 atm lifetime [yrs]',
                        'H2 soil sink lifetime [yrs]',
                        'H2 total lifetime [yrs]',]
df_budget_h2 = df_budget_h2.sort_index()

df_budget_h2



,H2 burden [Tg],H2 surf. conc [ppbv],H2 atm loss [Tg/yr],H2 atm prod [Tg/yr],H2 soil sink [Tg/yr],H2 estimated emissions [Tg/yr],H2 atm lifetime [yrs],H2 soil sink lifetime [yrs],H2 total lifetime [yrs]
CNTR,205,559,29.2,55.8,58.0,31.4,7.02,3.53,2.35
antro01,205,560,29.2,55.8,58.1,31.5,7.02,3.53,2.35
antro1,207,565,29.5,55.8,58.7,32.4,7.02,3.52,2.34
antro10,226,621,32.1,55.7,65.0,41.4,7.04,3.48,2.33
antro100,423,"1,189",58.0,54.9,128,131,7.28,3.30,2.27
epia,207,565,29.4,55.8,58.7,32.4,7.02,3.52,2.34
maud,207,568,29.5,55.8,58.7,32.4,7.02,3.53,2.35
maxdep,207,565,29.4,55.8,58.7,32.4,7.02,3.52,2.34
munich,207,565,29.5,55.8,58.7,32.4,7.02,3.52,2.34
nemo,207,567,29.5,55.8,58.7,32.4,7.02,3.53,2.35


In [257]:
df_budget_h2_ssp = pd.concat([df_h2_burden_ssp.iloc[0],
                              df_h2_surfconc_ssp.iloc[0],
                              df_h2_atmloss_ssp,
                              df_h2_atmprod_ssp,
                              df_h2_drydep_ssp,
                              df_h2_estemis_ssp, 
                              df_h2_atm_lifetime_ssp,
                              df_h2_soil_sink_lifetime_ssp,
                              df_h2_lifetime_ssp
                             ],axis=1)

df_budget_h2_ssp.columns = ['H2 burden [Tg]',
                            'H2 surf. conc [ppbv]',
                        'H2 atm loss [Tg/yr]',
                        'H2 atm prod [Tg/yr]',
                        'H2 soil sink [Tg/yr]',
                        'H2 estimated emissions [Tg/yr]',
                        'H2 atm lifetime [yrs]',
                        'H2 soil sink lifetime [yrs]',
                        'H2 total lifetime [yrs]',]
df_budget_h2_ssp = df_budget_h2_ssp.sort_index()
df_budget_h2_ssp

,H2 burden [Tg],H2 surf. conc [ppbv],H2 atm loss [Tg/yr],H2 atm prod [Tg/yr],H2 soil sink [Tg/yr],H2 estimated emissions [Tg/yr],H2 atm lifetime [yrs],H2 soil sink lifetime [yrs],H2 total lifetime [yrs]
10CH4_SSP119,188,514,25.1,47.2,53.4,31.4,7.48,3.52,2.39
10CH4_SSP434,235,639,28.6,63.4,66.1,31.2,8.21,3.55,2.48
10CH4_SSP585,245,667,31.4,69.0,68.9,31.2,7.81,3.56,2.44
CNTR_SSP119,181,495,24.9,45.0,51.5,31.4,7.26,3.51,2.37
CNTR_SSP434,225,613,28.3,60.4,63.5,31.3,7.95,3.54,2.45
CNTR_SSP585,234,638,31.0,65.6,65.9,31.3,7.55,3.55,2.41
antro10_SSP119,202,558,27.8,44.9,58.5,41.3,7.29,3.46,2.35
antro10_SSP434,247,677,31.0,60.3,70.6,41.2,7.97,3.50,2.43
antro10_SSP585,256,701,33.8,65.5,72.9,41.2,7.57,3.51,2.40


In [258]:
df_budget_h2 = pd.concat([df_budget_h2,df_budget_h2_ssp],sort=False)
df_budget_h2
#df_budget_h2.to_csv(outputpath + 'table_budget_h2.csv')

,H2 burden [Tg],H2 surf. conc [ppbv],H2 atm loss [Tg/yr],H2 atm prod [Tg/yr],H2 soil sink [Tg/yr],H2 estimated emissions [Tg/yr],H2 atm lifetime [yrs],H2 soil sink lifetime [yrs],H2 total lifetime [yrs]
CNTR,205,559,29.2,55.8,58.0,31.4,7.02,3.53,2.35
antro01,205,560,29.2,55.8,58.1,31.5,7.02,3.53,2.35
antro1,207,565,29.5,55.8,58.7,32.4,7.02,3.52,2.34
antro10,226,621,32.1,55.7,65.0,41.4,7.04,3.48,2.33
antro100,423,"1,189",58.0,54.9,128,131,7.28,3.30,2.27
epia,207,565,29.4,55.8,58.7,32.4,7.02,3.52,2.34
maud,207,568,29.5,55.8,58.7,32.4,7.02,3.53,2.35
maxdep,207,565,29.4,55.8,58.7,32.4,7.02,3.52,2.34
munich,207,565,29.5,55.8,58.7,32.4,7.02,3.52,2.34
nemo,207,567,29.5,55.8,58.7,32.4,7.02,3.53,2.35


In [259]:
df_budget_h2.loc[['CNTR','CNTR_SSP119','CNTR_SSP434','CNTR_SSP585']]

,H2 burden [Tg],H2 surf. conc [ppbv],H2 atm loss [Tg/yr],H2 atm prod [Tg/yr],H2 soil sink [Tg/yr],H2 estimated emissions [Tg/yr],H2 atm lifetime [yrs],H2 soil sink lifetime [yrs],H2 total lifetime [yrs]
CNTR,205,559,29.2,55.8,58.0,31.4,7.02,3.53,2.35
CNTR_SSP119,181,495,24.9,45.0,51.5,31.4,7.26,3.51,2.37
CNTR_SSP434,225,613,28.3,60.4,63.5,31.3,7.95,3.54,2.45
CNTR_SSP585,234,638,31.0,65.6,65.9,31.3,7.55,3.55,2.41


In [260]:
df_budget_h2.to_csv(outputpath + 'table_budget_h2.csv')

## CH4 budget table

In [204]:
df_ch4_loss_other_strat_conc.loc['CTRL']

20.729217388088895

In [205]:
budget_ch4 = np.array([df_ch4_burden_conc.loc['CTRL'],
                           df_ch4_surfconc_conc.loc['CTRL'],
                           df_ch4_loss_conc.loc['CTRL'],
                           df_ch4_loss_other_strat_conc.loc['CTRL'],
                           df_ch4_loss_soil_conc.loc['CTRL'],
                           df_ch4_loss_conc.loc['CTRL']+df_ch4_loss_other_strat_conc.loc['CTRL']+df_ch4_loss_soil_conc.loc['CTRL'],
                           df_ch4_lifetime_conc.loc['CTRL'],
                           df_ch4_tot_lifetime_conc.loc['CTRL']
                           ])

budget_ch4_columns = ['CH4 burden [Tg]','CH4 surface conc. [ppbv]',
                         'CH4 chem loss OH [Tg/yr]', 'CH4 chem loss other strat [Tg/yr]','CH4 loss soil [Tg/yr]',
                         'CH4 total loss [Tg/yr]',
                         'CH4 lifetime due to OH (whole atmosphere) [yrs]','Total CH4 lifetime [yrs]'] 

df_budget_ch4 = pd.DataFrame(data=budget_ch4,index=budget_ch4_columns,columns=['CTRL'])


In [206]:
temp =df_ch4_loss_ssp.loc[['CNTR_SSP119','CNTR_SSP434','CNTR_SSP585']]+df_ch4_loss_other_strat_ssp[['CNTR_SSP119','CNTR_SSP434','CNTR_SSP585']]+df_ch4_loss_soil_ssp[['CNTR_SSP119','CNTR_SSP434','CNTR_SSP585']]
temp.iloc[0]

CNTR_SSP119   552
CNTR_SSP434   787
CNTR_SSP585   910
Name: YR6, dtype: float64

In [207]:
df_ch4_burden_ssp[['CNTR_SSP119','CNTR_SSP434','CNTR_SSP585']]

,CNTR_SSP119,CNTR_SSP434,CNTR_SSP585
Scenario,,,
YR6,"3,916","6,104","6,713"
deltaCH4,0.00,0.00,0.00


In [208]:
budget_ch4_ssp = np.array([df_ch4_burden_ssp[['CNTR_SSP119','CNTR_SSP434','CNTR_SSP585']].iloc[0].values,
                           df_ch4_surfconc_ssp[['CNTR_SSP119','CNTR_SSP434','CNTR_SSP585']].iloc[0].values,
                           df_ch4_loss_ssp.loc[['CNTR_SSP119','CNTR_SSP434','CNTR_SSP585']].values,
                           df_ch4_loss_other_strat_ssp[['CNTR_SSP119','CNTR_SSP434','CNTR_SSP585']].iloc[0].values,
                           df_ch4_loss_soil_ssp[['CNTR_SSP119','CNTR_SSP434','CNTR_SSP585']].iloc[0].values,
                           temp.iloc[0].values,
                           df_ch4_lifetime_ssp[['CNTR_SSP119','CNTR_SSP434','CNTR_SSP585']].iloc[0].values,
                           df_ch4_tot_lifetime_ssp[['CNTR_SSP119','CNTR_SSP434','CNTR_SSP585']].iloc[0].values
                           ])
print(budget_ch4_ssp)

budget_ch4_columns = ['CH4 burden [Tg]','CH4 surface conc. [ppbv]',
                        'CH4 chem loss OH [Tg/yr]', 'CH4 chem loss other strat [Tg/yr]','CH4 loss soil [Tg/yr]',
                         'CH4 total loss [Tg/yr]',
                         'CH4 lifetime due to OH (whole atmosphere) [yrs]','Total CH4 lifetime [yrs]'] 

df_budget_ch4_ssp = pd.DataFrame(data=budget_ch4_ssp,index=budget_ch4_columns,columns=['CNTR_SSP119','CNTR_SSP434','CNTR_SSP585'])
df_budget_ch4_ssp.T

[[3916.26457088 6104.02190404 6712.63708501]
 [1426.87834394 2222.81176741 2445.8181542 ]
 [ 510.92725654  723.76513611  840.51601313]
 [  16.31776905   25.4334246    27.96932119]
 [  24.47665357   38.1501369    41.95398178]
 [ 551.72167916  787.34869761  910.4393161 ]
 [   7.66501399    8.43370535    7.98632861]
 [   7.09826117    7.75262844    7.37296486]]


,CH4 burden [Tg],CH4 surface conc. [ppbv],CH4 chem loss OH [Tg/yr],CH4 chem loss other strat [Tg/yr],CH4 loss soil [Tg/yr],CH4 total loss [Tg/yr],CH4 lifetime due to OH (whole atmosphere) [yrs],Total CH4 lifetime [yrs]
CNTR_SSP119,"3,916","1,427",511,16.3,24.5,552,7.67,7.10
CNTR_SSP434,"6,104","2,223",724,25.4,38.2,787,8.43,7.75
CNTR_SSP585,"6,713","2,446",841,28.0,42.0,910,7.99,7.37


In [209]:
df_budget_ch4 = pd.concat([df_budget_ch4,df_budget_ch4_ssp],axis=1,sort=False)
df_budget_ch4.T.to_csv(outputpath + 'table_budget_ch4.csv')

df_budget_ch4.T

,CH4 burden [Tg],CH4 surface conc. [ppbv],CH4 chem loss OH [Tg/yr],CH4 chem loss other strat [Tg/yr],CH4 loss soil [Tg/yr],CH4 total loss [Tg/yr],CH4 lifetime due to OH (whole atmosphere) [yrs],Total CH4 lifetime [yrs]
CTRL,"4,975","1,813",674,20.7,31.1,726,7.38,6.85
CNTR_SSP119,"3,916","1,427",511,16.3,24.5,552,7.67,7.10
CNTR_SSP434,"6,104","2,223",724,25.4,38.2,787,8.43,7.75
CNTR_SSP585,"6,713","2,446",841,28.0,42.0,910,7.99,7.37


In [210]:
#Write AGWP values to file
df_h2_agwp  = pd.concat([df_h2_agwp_ch4,
                         df_h2_agwp_o3,
                         df_h2_agwp_h2o,
                         df_h2_agwp_ch4ind_o3,
                         df_h2_agwp_ch4ind_h2o],axis=1,sort=False)

df_h2_agwp.to_csv(outputpath + 'table_h2_agwp.csv') 
df_h2_agwp

,h2_agwp_ch4,h2_agwp_o3,0,h2_agwp_ch4ind_o3,h2_agwp_ch4ind_h2o
CNTR,nan,nan,nan,nan,NaN
antro01,0.43,0.19,0.17,0.25,0.06
antro1,0.43,0.21,0.17,0.25,0.06
antro10,0.43,0.22,0.17,0.25,0.06
antro100,0.41,0.22,0.17,0.24,0.06
nemo,0.50,0.23,0.18,0.29,0.07
epia,0.39,0.18,0.14,0.23,0.05
munich,0.41,0.20,0.15,0.24,0.06
usdrydep,0.36,0.17,0.13,0.21,0.05
maud,0.50,0.23,0.18,0.29,0.07


# Appendix:

## Methane feedback factor:

### Atmospheric mass conversion CH4  [Tg/ppb] (from perturbations)

In [211]:
df_ch4_burden_per_conc  = df_ch4_burden_conc.loc['deltaCH4']/df_ch4_surfconc_conc.loc['deltaCH4']
#df_ch4_burden_per_conc.name = 'ch4_burden_per_conc'
df_ch4_burden_per_conc

2.7494761488687924

In [212]:
df_ch4_surfconc_ssp

,10CH4_SSP119,10CH4_SSP434,10CH4_SSP585,CNTR_SSP119,CNTR_SSP434,CNTR_SSP585,antro10_SSP119,antro10_SSP434,antro10_SSP585
YR6,"1,570","2,445","2,690","1,427","2,223","2,446","1,427","2,223","2,446"
delta,143,222,245,0.00,0.00,0.00,0.00,0.00,0.00


In [213]:
df_ch4_burden_per_conc_ssp  = df_ch4_burden_ssp.loc['deltaCH4']/df_ch4_surfconc_ssp.loc['delta']
#df_ch4_burden_per_conc.name = 'ch4_burden_per_conc'
df_ch4_burden_per_conc_ssp

10CH4_SSP119        2.75
10CH4_SSP434        2.75
10CH4_SSP585        2.75
CNTR_SSP119          nan
CNTR_SSP434          nan
CNTR_SSP585          nan
antro10_SSP119   247,503
antro10_SSP434   101,333
antro10_SSP585    88,398
dtype: float64

### Increase per unit flux w/o feedback = integrated decay [ppb yr/Tg]

In [214]:
df_w_o_feedback =df_ch4_tot_lifetime_conc.loc['CTRL']/df_ch4_burden_per_conc #Lifetime [yr] / [Tg/ppb] 
#df_w_o_feedback.name = 'increase_w_o_feedback'
df_w_o_feedback_to_file = pd.DataFrame(data=[df_w_o_feedback],index=['OSLOCTM3'])
df_w_o_feedback_to_file.to_csv(outputpath + 'increase_w_o_feedback.csv')
df_w_o_feedback

2.4923974591529294

In [215]:
df_w_o_feedback_ssp =df_ch4_tot_lifetime_ssp/df_ch4_burden_per_conc_ssp #Lifetime [yr] / [Tg/ppb] 
df_w_o_feedback_ssp[['10CH4_SSP119','10CH4_SSP434','10CH4_SSP585']]

##df_w_o_feedback.name = 'increase_w_o_feedback'
df_w_o_feedback_to_file_ssp = pd.DataFrame(data=[df_w_o_feedback_ssp[['10CH4_SSP119','10CH4_SSP434','10CH4_SSP585']]],index=['OSLOCTM3'])

df_w_o_feedback_ssp = df_w_o_feedback_ssp[['10CH4_SSP119','10CH4_SSP434','10CH4_SSP585']]
df_w_o_feedback_ssp.columns = ['SSP119','SSP434','SSP585']
df_w_o_feedback_to_file_ssp.to_csv(outputpath + 'increase_w_o_feedback_ssp.csv')
df_w_o_feedback_ssp

,SSP119,SSP434,SSP585
Scenario,,,
YR6,2.66,2.91,2.77


### Feedback factor: increase CH4 with feedback/ increase CH4 without feedback

In [216]:
df_feedback_factor_ch4 = df_surf_ch4_per_ch4_flux_conc/df_w_o_feedback
#df_feedback_factor_ch4.name = 'feedback_factor_ch4'
df_feedback_to_file = pd.DataFrame(data=[df_feedback_factor_ch4],index=['CTRL'])
#df_feedback_to_file.to_csv(outputpath + 'feedback_factor_ch4.csv')
df_feedback_factor_ch4

1.4645748747886003

In [217]:
df_feedback_factor_ch4_ssp = df_surf_ch4_per_ch4_flux_ssp/df_w_o_feedback

##df_feedback_factor_ch4.name = 'feedback_factor_ch4'
df_feedback_to_file_ssp = pd.DataFrame(data=[df_feedback_factor_ch4_ssp],index=['CTRL'])
#df_feedback_to_file_ssp.to_csv(outputpath + 'feedback_factor_ch4_ssp.csv')
df_feedback_factor_ch4_ssp
#df_feedback_to_file_ssp

SSP119   1.52
SSP434   1.71
SSP585   1.67
Name: surf_ch4_per_ch4_flux, dtype: object

In [218]:
df_feedback_factor_ch4_ssp.index

Index(['SSP119', 'SSP434', 'SSP585'], dtype='object')

In [219]:
df_feedback_factor_ch4_comb =  pd.DataFrame(data=[],index=['OSLOCTM'],columns=df_h2_gwp.columns)
df_feedback_factor_ch4_comb[['antro01','antro1','antro10','antro100','nemo','epia','munich','usdrydep','maud','zep','maxdep']]=df_feedback_factor_ch4
#df_feedback_factor_ch4_comb[df_feedback_factor_ch4_ssp.index] = df_feedback_factor_ch4_ssp[df_feedback_factor_ch4_ssp.index]
scen_list = ['SSP119', 'SSP434', 'SSP585']
for scen in scen_list:
    df_feedback_factor_ch4_comb[scen]=df_feedback_factor_ch4_ssp[scen]

df_feedback_factor_ch4_comb.to_csv(outputpath + 'feedback_factor_ch4.csv')
df_feedback_factor_ch4_comb

,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep,SSP119,SSP434,SSP585
OSLOCTM,1.46,1.46,1.46,1.46,1.46,1.46,1.46,1.46,1.46,1.46,1.46,1.52,1.71,1.67


Split the CH4 GWP into direct and indirect based on the feedback factor.

In [220]:
df_feedback_frac = 1.0 - (1.0/df_feedback_factor_ch4_comb)
df_feedback_frac

,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep,SSP119,SSP434,SSP585
OSLOCTM,0.32,0.32,0.32,0.32,0.32,0.32,0.32,0.32,0.32,0.32,0.32,0.34,0.42,0.40


feedback_factor = df_feedback_factor_ch4
feedback_frac = 1.0 - (1.0/feedback_factor)
#feedback_frac.name = 'feedback_frac'
feedback_frac

feedback_factor_ssp = df_feedback_factor_ch4_ssp
feedback_frac_ssp = 1.0 - (1.0/feedback_factor_ssp)
#feedback_frac.name = 'feedback_frac'
feedback_frac_ssp

df_feedback_frac =  pd.DataFrame(data=[],index=['OSLOCTM'],columns=df_h2_gwp.columns)
df_feedback_frac[['antro01','antro1','antro10','antro100','nemo','epia','munich','usdrydep','maud','zep','maxdep']]=feedback_frac
df_feedback_frac[df_feedback_factor_ch4_ssp.index] = feedback_frac_ssp.values
df_feedback_frac

In [221]:
#Save to file:

df_h2_gwp.loc['CH4dir'] = df_h2_gwp.loc['CH4']*(1.0-df_feedback_frac.loc['OSLOCTM'])
df_h2_gwp.loc['CH4indir'] = df_h2_gwp.loc['CH4']*df_feedback_frac.loc['OSLOCTM']

#df_h2_gwp['GFDL-emi'].loc['CH4dir'] = df_h2_gwp['GFDL-emi'].loc['CH4']
#df_h2_gwp['OSLOCTM3-emi'].loc['CH4dir'] = df_h2_gwp['OSLOCTM3-emi'].loc['CH4']
df_h2_gwp = df_h2_gwp.drop(['total','CH4'])
df_h2_gwp.to_csv(outputpath + 'table_h2_gwp.csv')

df_h2_gwp

,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep,SSP119,SSP434,SSP585
O3,2.13,2.38,2.42,2.41,2.59,2.06,2.22,1.89,2.62,2.14,2.27,2.49,2.06,2.02
strat H2O,1.87,1.87,1.88,1.90,2.05,1.57,1.69,1.45,2.05,1.65,1.80,1.88,1.99,1.99
O3 CH4ind,2.79,2.81,2.80,2.66,3.25,2.53,2.70,2.32,3.25,2.67,2.58,2.94,2.79,3.03
strat H2O CH4ind,0.65,0.66,0.66,0.62,0.76,0.59,0.63,0.54,0.76,0.62,0.60,0.96,1.09,1.15
CH4dir,3.26,3.29,3.28,3.11,3.80,2.95,3.16,2.71,3.80,3.12,3.02,3.18,3.18,3.44
CH4indir,1.52,1.53,1.52,1.45,1.76,1.37,1.47,1.26,1.77,1.45,1.40,1.64,2.27,2.32


In [222]:
df_h2_gwp.loc['CH4indir']/(df_h2_gwp.loc['CH4dir']+df_h2_gwp.loc['CH4indir'])

antro01    0.32
antro1     0.32
antro10    0.32
antro100   0.32
nemo       0.32
epia       0.32
munich     0.32
usdrydep   0.32
maud       0.32
zep        0.32
maxdep     0.32
SSP119     0.34
SSP434     0.42
SSP585     0.40
dtype: float64

In [223]:
df_feedback_factor_ch4

1.4645748747886003

In [224]:
df_feedback_factor_ch4_ssp

SSP119   1.52
SSP434   1.71
SSP585   1.67
Name: surf_ch4_per_ch4_flux, dtype: object

In [225]:
#Alternative feedback calc:
#ss the sensitivity of the lifetime to the burden.

#ss = 1 – dln[<k All><H2>]/dln[<H2>] 
#= 1 – dln[<k All>]/dln[<H2>] – dln[<H2>]/dln[<H2>] 
#= – dln[<k All>]/dln[<H2>] 
#=  +dln[1/<k All>] / dln[<H2>] 

#<k All> = <k OH> + <kstrat OH> + <k soil> Are the inverse liftime-
k_all_inv = df_ch4_tot_lifetime_conc

a = np.log(k_all_inv.loc['10CH4'])-np.log(k_all_inv.loc['CTRL'])

b = np.log(df_ch4_burden_conc.loc['10CH4'])-np.log(df_ch4_burden_conc.loc['CTRL'])
#print(b)
ss = a/b
ff = 1/(1-ss)
print(ff)

1.4429794488833552


In [226]:
ss = (np.log(df_ch4_tot_lifetime_conc.loc['10CH4']/df_ch4_tot_lifetime_conc.loc['CTRL']))/np.log(df_ch4_burden_conc.loc['10CH4']/df_ch4_burden_conc.loc['CTRL'])
ff=1/(1-ss)
print(ff)

1.4429794488833612


## Hydrogen feedback factor:

### Atmospheric mass conversion H2  [Tg/ppb] (from perturbations)

In [227]:
df_h2_burden_per_conc  = df_h2_burden.loc['deltaH2']/df_h2_surfconc.loc['delta']
df_h2_burden_per_conc.name = 'h2_burden_per_conc'
df_h2_burden_per_conc

CNTR        nan
antro01    0.35
antro1     0.35
antro10    0.35
antro100   0.35
nemo       0.32
epia       0.32
munich     0.33
usdrydep   0.34
maud       0.28
zep        0.31
maxdep     0.36
Name: h2_burden_per_conc, dtype: float64

In [228]:
df_h2_burden_per_conc_ssp  = df_h2_burden_ssp.loc['deltaH2']/df_h2_surfconc_ssp.loc['delta']
df_h2_burden_per_conc_ssp.name = 'h2_burden_per_conc'
df_h2_burden_per_conc_ssp

10CH4_SSP119     0.37
10CH4_SSP434     0.37
10CH4_SSP585     0.37
CNTR_SSP119       nan
CNTR_SSP434       nan
CNTR_SSP585       nan
antro10_SSP119   0.35
antro10_SSP434   0.35
antro10_SSP585   0.34
Name: h2_burden_per_conc, dtype: float64

### Increase per unit flux w/o feedback = integrated decay [ppb yr/Tg]

In [229]:
df_w_o_feedback_h2 =df_h2_lifetime.loc['CNTR']/df_h2_burden_per_conc #Lifetime [yr] / [Tg/ppb] 
df_w_o_feedback_h2

CNTR        nan
antro01    6.78
antro1     6.78
antro10    6.78
antro100   6.78
nemo       7.27
epia       7.30
munich     7.13
usdrydep   6.88
maud       8.39
zep        7.52
maxdep     6.45
Name: h2_burden_per_conc, dtype: float64

In [230]:
df_w_o_feedback_h2_ssp =df_h2_lifetime_ssp/df_h2_burden_per_conc_ssp #Lifetime [yr] / [Tg/ppb] 
#df_w_o_feedback_h2_ssp[['antro10_SSP119','antro10_SSP434','antro10_SSP585']]

###df_w_o_feedback.name = 'increase_w_o_feedback'
#df_w_o_feedback_h2_to_file_ssp = pd.DataFrame(data=[df_w_o_feedback_ssp[['10CH4_SSP119','10CH4_SSP434','10CH4_SSP585']]],index=['OSLOCTM3'])
#
#df_w_o_feedback_h2_ssp = df_w_o_feedback_h2_ssp[['antro10_SSP119','antro10_SSP434','antro10_SSP585']]
#df_w_o_feedback_h2_ssp.columns = ['SSP119','SSP434','SSP585']

scen_dict = {'antro10_SSP119':'SSP119',
             'antro10_SSP434':'SSP434',
             'antro10_SSP585':'SSP585'}

df_w_o_feedback_h2_ssp =df_w_o_feedback_h2_ssp[['antro10_SSP119','antro10_SSP434','antro10_SSP585']]

df_w_o_feedback_h2_ssp.index = ['SSP119','SSP434','SSP585']


df_w_o_feedback_h2_ssp

SSP119   6.80
SSP434   7.05
SSP585   6.95
dtype: float64

### Feedback factor: increase H2 with feedback/ increase H2 without feedback

If you take the control lifetime from the budget terms and divide it into the lifetime of the perturbation (the added burden from the 10% increase divided by the change in chemical flux from that perturbation.) You should get the feed back factor.

In [231]:
df_feedback_factor_h2 = df_surf_h2_per_h2_flux/df_w_o_feedback_h2
df_feedback_factor_h2.name = 'feedback_factor_h2'
#df_feedback_factor_h2.to_csv(outputpath + 'feedback_factor_h2.csv')
df_feedback_factor_h2
#Fabien wrote in the paper about feedback factor less than 1.

CNTR        nan
antro01    0.92
antro1     0.92
antro10    0.93
antro100   0.94
nemo       1.09
epia       0.83
munich     0.89
usdrydep   0.76
maud       1.10
zep        0.89
maxdep     0.85
Name: feedback_factor_h2, dtype: float64

In [232]:
df_surf_h2_per_h2_flux_ssp_v2 = df_surf_h2_per_h2_flux_ssp[['antro10_SSP119','antro10_SSP434','antro10_SSP585']]
df_surf_h2_per_h2_flux_ssp_v2.index = ['SSP119','SSP434','SSP585']
df_surf_h2_per_h2_flux_ssp_v2

SSP119   6.36
SSP434   6.52
SSP585   6.42
Name: surf_h2_per_h2_flux, dtype: float64

In [233]:
df_feedback_factor_h2_ssp = df_surf_h2_per_h2_flux_ssp_v2/df_w_o_feedback_h2_ssp

##df_feedback_factor_ch4.name = 'feedback_factor_ch4'
#df_feedback_to_file_ssp = pd.DataFrame(data=[df_feedback_factor_ch4_ssp],index=['CTRL'])
#df_feedback_to_file_ssp.to_csv(outputpath + 'feedback_factor_ch4_ssp.csv')
df_feedback_factor_h2_ssp
#df_feedback_to_file_ssp

SSP119   0.93
SSP434   0.93
SSP585   0.92
dtype: float64

In [234]:
df_feedback_factor_h2_comb =  pd.DataFrame(data=[],index=['OSLOCTM'],columns=df_h2_gwp.columns)
for scen in df_feedback_factor_h2.drop('CNTR').index:
    df_feedback_factor_h2_comb[scen] = df_feedback_factor_h2[scen]
df_feedback_factor_h2_comb

,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep,SSP119,SSP434,SSP585
OSLOCTM,0.92,0.92,0.93,0.94,1.09,0.83,0.89,0.76,1.10,0.89,0.85,NaN,NaN,NaN


In [235]:
scen_list = ['SSP119', 'SSP434', 'SSP585']
for scen in scen_list:
    df_feedback_factor_h2_comb[scen]=df_feedback_factor_h2_ssp[scen]

df_feedback_factor_h2_comb.to_csv(outputpath + 'feedback_factor_h2.csv')
df_feedback_factor_h2_comb

,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep,SSP119,SSP434,SSP585
OSLOCTM,0.92,0.92,0.93,0.94,1.09,0.83,0.89,0.76,1.10,0.89,0.85,0.93,0.93,0.92


### Change in lifetime per flux

In [236]:
df_ch4_lifetime.loc['deltaH2'] = df_ch4_lifetime.iloc[0]-df_ch4_lifetime['CNTR'].iloc[0]
df_ch4_lifetime.loc['deltaCH4'] = df_ch4_lifetime_conc.loc['10CH4']-df_ch4_lifetime_conc.loc['CTRL']
df_ch4_lifetime

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
YR13,7.39,7.39,7.40,7.43,7.73,7.40,7.40,7.40,7.40,7.40,7.40,7.40
deltaH2,0.00,0.00,0.00,0.03,0.33,0.00,0.00,0.00,0.00,0.00,0.00,0.00
deltaCH4,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24


In [237]:
#Direct (changes in methane lifetime per h2 flux [days per Tg H2])
df_ch4_lifetime_per_h2_flux =  df_ch4_lifetime.loc['deltaH2']/df_h2_flux['deltaH2']
df_ch4_lifetime_per_h2_flux*365.0 #Days

CNTR        nan
antro01    1.22
antro1     1.23
antro10    1.23
antro100   1.22
nemo       1.42
epia       1.11
munich     1.19
usdrydep   1.02
maud       1.42
zep        1.17
maxdep     1.13
Name: deltaH2, dtype: float64

In [238]:
#Indirect (changes in methane lifetime per h2 flux [days per Tg H2] due to changes in methane):
df_ch4_lifetime_per_ch4_flux =  df_ch4_lifetime.loc['deltaCH4']/df_ch4_flux.loc['deltaCH4']
df_ch4_lifetime_per_ch4_flux*365.0*df_ch4_flux_per_h2_flux


CNTR        nan
antro01    0.53
antro1     0.53
antro10    0.53
antro100   0.51
nemo       0.62
epia       0.48
munich     0.51
usdrydep   0.44
maud       0.62
zep        0.51
maxdep     0.49
dtype: float64

### Check that delta flux equal delta burden divided by lifetime including the feedback effect

In [239]:
df_feedback_factor_h2

CNTR        nan
antro01    0.92
antro1     0.92
antro10    0.93
antro100   0.94
nemo       1.09
epia       0.83
munich     0.89
usdrydep   0.76
maud       1.10
zep        0.89
maxdep     0.85
Name: feedback_factor_h2, dtype: float64

In [240]:
df_h2_burden

,CNTR,antro01,antro1,antro10,antro100,nemo,epia,munich,usdrydep,maud,zep,maxdep
Scenario,,,,,,,,,,,,
YR13,205,205,207,226,423,207,207,207,206,207,207,207
deltaH2,0.00,0.22,2.15,21.5,218,2.53,1.93,2.05,1.76,2.56,2.07,1.97


In [241]:
ss = (np.log(df_h2_lifetime/df_h2_lifetime.loc['CNTR']))/np.log(df_h2_burden.iloc[0]/df_h2_burden['CNTR'].iloc[0])
ff=1/(1-ss)
print(ff)

CNTR        nan
antro01    0.92
antro1     0.93
antro10    0.93
antro100   0.96
nemo       1.09
epia       0.83
munich     0.89
usdrydep   0.76
maud       1.10
zep        0.89
maxdep     0.85
dtype: float64


In [242]:
#Delta burden divided by tau*ff
test = df_h2_burden.loc['deltaH2']/(df_h2_lifetime.loc['CNTR']*df_feedback_factor_h2)
print(test)
print(df_h2_flux['deltaH2'])


CNTR        nan
antro01    0.10
antro1     0.99
antro10    9.90
antro100   99.1
nemo       0.99
epia       0.99
munich     0.99
usdrydep   0.99
maud       0.99
zep        0.99
maxdep     0.99
dtype: float64
CNTR       0.00
antro01    0.10
antro1     0.99
antro10    9.90
antro100   99.1
nemo       0.99
epia       0.99
munich     0.99
usdrydep   0.99
maud       0.99
zep        0.99
maxdep     0.99
Name: deltaH2, dtype: float64


In [243]:
(df_ch4_tot_lifetime.iloc[0]['CNTR']*df_feedback_factor_ch4)

10.055132897458343

In [244]:
test = df_ch4_burden.loc['deltaCH4']/(df_ch4_tot_lifetime.iloc[0]['CNTR']*df_feedback_factor_ch4)
print(test)
print(df_ch4_flux.loc['deltaCH4'])

CNTR       0.00
antro01    0.00
antro1     0.00
antro10    0.00
antro100   0.04
nemo       0.00
epia       0.00
munich     0.00
usdrydep   0.00
maud       0.00
zep        0.00
maxdep     0.00
Name: deltaCH4, dtype: float64
CNTR       49.7
antro01    49.7
antro1     49.7
antro10    49.7
antro100   49.7
nemo       49.7
epia       49.7
munich     49.7
usdrydep   49.7
maud       49.7
zep        49.7
maxdep     49.7
Name: deltaCH4, dtype: float64


## Feedback factor summary

The feedback factor calculation employed above is equivalent with a division between the perturbation lifetime and the total lifetime, where the perturbation lifetime is defined as the burden change diveded by the flux change in the perturbed simulation relative to the control simulation

In [245]:
print(df_feedback_factor_ch4)
test_ff =  (df_ch4_burden_conc.loc['deltaCH4']/df_ch4_flux_conc.loc['deltaCH4'])/df_ch4_tot_lifetime_conc.loc['CTRL']

test_ff

1.4645748747886003


1.4645748747886005

In [246]:
print(df_feedback_factor_h2)
test_ff =  (df_h2_burden.loc['deltaH2']/df_h2_flux['deltaH2'])/df_h2_lifetime.loc['CNTR']

test_ff



CNTR        nan
antro01    0.92
antro1     0.92
antro10    0.93
antro100   0.94
nemo       1.09
epia       0.83
munich     0.89
usdrydep   0.76
maud       1.10
zep        0.89
maxdep     0.85
Name: feedback_factor_h2, dtype: float64


CNTR        nan
antro01    0.92
antro1     0.92
antro10    0.93
antro100   0.94
nemo       1.09
epia       0.83
munich     0.89
usdrydep   0.76
maud       1.10
zep        0.89
maxdep     0.85
Name: deltaH2, dtype: float64

## Calculating values to be used for adding composition changes from the methan run

In [247]:
df_per_flux_h2.loc['antro10']

Flux H2 [Tg/yr]                          9.90
Surf. conc. H2 per flux [ppb yr/Tg]      6.28
Surf. conc. CH4 per flux [ppb yr/Tg]     1.12
Flux CH4/Flux H2 [Tg CH4/Tg H2]          0.31
CH4 RF per flux [mW m-2 yr/ Tg]          0.43
ozone RF per flux [mW m-2 yr/ Tg]        0.22
Strat. H2O RF per flux [mW m-2 yr/ Tg]   0.17
Aerosol RF per flux [mW m-2 yr/ Tg]       nan
Name: antro10, dtype: float64

In [248]:
df_per_flux_ch4.loc['antro10']

Flux CH4 [Tg/yr]                        49.7
Surf. conc. CH4 per flux [ppb yr/Tg]    3.65
Flux H2/Flux CH4 [Tg H2/Tg CH4]         0.02
ozone RF per flux [mW m-2 yr/ Tg]       0.82
Strat H2O RF per flux [mW m-2 yr/ Tg]   0.19
Aerosol RF per flux [mW m-2 yr/ Tg]      NaN
Name: antro10, dtype: object

This is based on the 10% increase in surface methane concentration

Fraction of changes in atmospheric composition to be added to the changes in the pure hydrogen perturbation. To include methane induced changes.

  [Tg CH4/Tg H2]*[Tg H2/yr]/[Tg CH4/yr]

df_per_flux_h2['Flux CH4/Flux H2 [Tg CH4/Tg H2]'].loc['antro10']

*df_per_flux_h2['Flux H2 [Tg/yr]'].loc['antro10']

/df_per_flux_ch4['Flux CH4 [Tg/yr]'].loc['antro10']

In [249]:
frac = df_per_flux_h2['Flux CH4/Flux H2 [Tg CH4/Tg H2]'].loc['antro10']*df_per_flux_h2['Flux H2 [Tg/yr]'].loc['antro10']/df_per_flux_ch4['Flux CH4 [Tg/yr]'].loc['antro10']
frac

0.06093482460432349